In [1]:
import os
import pickle
import string
import argparse

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1

REBUILDDICTIONARYFROMPICKLEFILE = 0

path = 'F:\LaTeX\LatexWork\Bible'
os.chdir(path)
print (os.getcwd())

import plotly

F:\LaTeX\LatexWork\Bible


In [2]:
# %load BibleWordList2.py
'''
Update 21 Deceber 2018, 0004
'''


import os
import pickle
import collections
import operator
import re

DEBUG = 0

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1
REBUILDDICTIONARYFROMPICKLEFILE = 0

punctuations = '''!()_;:,<>?@#$^.'''

exceptions = [
    'A', 'Accept', 'After', 'Aha', 'All','Alas','Also','Arise',
    'Amen', 'An', 'And', 'As', 'At', 'Awake','According',
    'Be', 'Because', 'Before', 'Behold', 'Blessed','Build','Better',
    'Blessings', 'Blind', 'Both', 'Bring', 'But','Beautiful',
    'By', 'Command', 'Concerning', 'Consider', 'Cut','Come',"Children's",
    'Deal', 'Deceit', 'Deep', 'Deliver', 'Depart','Did',
    'Doubtless', 'Each', 'Either', 'Even', 'Every','Excellent','Fear',
    'For', 'From', 'Get', 'Give', 'Go', 'Grace',
    'Great', 'Hatred', 'Having', 'He', 'Hear',
    'Hearing', 'Heaviness', 'Help', 'His', 'Hold',
    'Horror', 'How', 'If', 'In', 'Incline', 'Judge',
    'Leave', 'Let', 'Lo', 'Look', 'Lying', 'Lodge','Make','Must','Mark',
    'Many', 'Mine', 'My', 'Neither', 'Nevertheless','Nay',
    'Not', 'Now', 'Number', 'O', 'Of', 'On', 'One','Out',
    'Only', 'Open', 'Or', 'Order', 'Our', 'Pardon',
    'Plead', 'Princes', 'Quicken', 'Red', 'Remember',
    'Remove', 'Riches', 'Righteous', 'Rise', 'Rivers','Return',
    'Sacrifice', 'Salvation', 'Say', 'Seven', 'Shall','Spake',
    'So', 'Speak', 'Sprinkle', 'Stablish', 'Stand','Strong',
    'Surely', 'Take', 'Teach', 'That', 'The', 'Their',
    'Then', 'There', 'Therefore', 'These', 'They',
    'This', 'Those', 'Thou', 'Though', 'Through',
    'Thus', 'Thy', 'To', 'Treasures', 'Trouble','Told',
    'Turn', 'Two', 'Unless', 'Uphold', 'We',
    'What', 'Whatsoever', 'When', 'Where', 'Wherefore',
    'Wherewithal', 'Which', 'Who', 'Whom', 'Whoso',
    'Whosoever', 'Why', 'Wise', 'With', 'Withhold','Walk','Wisdom',
    'Would', 'Ye', 'Yea', 'Yet', 'Your']

properNameList = [
    'Aaron', 'Abelmizraim', 'Abiasaph', 'Abib', 'Abihu', 'Abraham', 'Abram',
    'Aceldama', 'Achaia', 'Achaicus', 'Achor', 'Achzib', 'Adam', 'Adramyttium',
    'Adria', 'Adullam', 'Agrippa', 'Ahab', 'Ahaz', 'Ahikam', 'Ahisamach',
    'Aholiab', 'Alexandria', 'Alexandrians', 'Alphaeus', 'Amalek', 'Amittai',
    'Ammi ', 'Amminadab', 'Amon', 'Amorite', 'Amorites', 'Amram', 'Anathoth',
    'Andrew', 'Angel', 'Antioch', 'Aphrah', 'Apollos', 'Apphia', 'Aquila',
    'Arabia ', 'Arabian', 'Archippus', 'Ard', 'Areli', 'Aretas', 'Aristarchus',
    'Arnon', 'Arodi', 'Aroer ', 'Asenath', 'Asenath', 'Ashbel', 'Asher',
    'Ashkelon', 'Asia', 'Assir', 'Assyria ', 'Assyrian', 'Atad', 'Augustus',
    'Azariah', 'Azur', 'BRANCH', 'Baal ', 'Baali ', 'Baalim', 'Baalzephon',
    'Bablyon', 'Balaam', 'Balak', 'Barbarian', 'Barjona', 'Barnabas',
    'Barsabas', 'Bartholomew', 'Baruch', 'Becher', 'Beeri', 'Beerlahairoi',
    'Beersheba', 'Belah', 'Belial', 'Belteshazzar', 'Benjamin', 'Beor',
    'Bered', 'Beriah', 'Bernice', 'Bethaven', 'Bethdiblathaim ', 'Bethel',
    'Bethezel', 'Bethgamul', 'Bethlehem', 'Bethmeon', 'Bethshemesh',
    'Bezaleel', 'Bilhah', 'Bozrah', 'Caesar ', 'Caesarea', 'Cainan', 'Cana',
    'Canaan', 'Canaanite', 'Canaanites', 'Canaanitish', 'Capernaum', 'Caphtor',
    'Carchemish', 'Carmi', 'Castor', 'Cephas', 'Chaldeans', 'Chaldeans',
    'Chemosh', 'Chittim', 'Chloe', 'Christ', 'Cilicia', 'Clauda', 'Cnidus',
    'Crete', 'Cretians', 'Crispus', 'Cyprus', 'Cyrenians', 'Cyrus',
    'Damascenes', 'Damascus', 'Dan', 'Daniel', 'Darius', 'David', 'Demas',
    'Diblaim', 'Dibon', 'Didymus', 'Dinah', 'Edom', 'Egypt', 'Egyptian',
    'Egyptians', 'Ehi', 'Elasah', 'Eleazar', 'Elias', 'Eliezer', 'Elim',
    'Elisheba', 'Elkanah', 'Elon', 'Elzaphan', 'Enoch', 'Enos', 'Epaphras',
    'Ephesus', 'Ephraim', 'Ephrath', 'Ephron', 'Er', 'Eri', 'Esaias', 'Esau',
    'Etham', 'Euphrates', 'Euroclydon', 'Ezbon', 'Father', 'Felix', 'Festus',
    'Fortunatus', 'Gad', 'Gadarenes', 'Gaius', 'Galatia', 'Galilee', 'Gath',
    'Gaza', 'Gedaliah', 'Gemariah', 'Gentiles', 'Gera', 'Gershom', 'Gershon',
    'Gibeah', 'Gibeon', 'Gilead', 'Gilgal', 'God', 'Gomer', 'Goshen',
    'Grecians', 'Greek', 'Guni', 'Hagar', 'Haggi', 'Ham', 'Hananiah', 'Hanoch',
    'Heber', 'Hebrew', 'Hebrews', "Hebrews'", 'Hebron', 'Heldai', 'Helem',
    'Hen', 'Heshbon', 'Hezekiah', 'Hezron', 'High', 'Hilkiah', 'Hinnom',
    'Hittite', 'Hittites', 'Hivite', 'Hivites', 'Horeb', 'Horonaim', 'Hosea',
    'Hoshaiah', 'Huppim', 'Hur', 'Hushim', 'I', 'Isaac', 'Ishi', 'Ishmael',
    'Ishmeelites', 'Ishuah', 'Israel', 'Issachar', 'Issachar', 'Isui', 'Italy',
    'Ithamar', 'Ithiel', 'Izhar', 'JEHOVAH', 'Jachin', 'Jacob', 'Jahleel',
    'Jahzeel', 'Jairus', 'Jakeh', 'James', 'Jamin', 'Japheth', 'Jareb:',
    'Jared', 'Jebusite', 'Jebusites', 'Jeconiah', 'Jedaiah', 'Jehoiada',
    'Jehoiakim', 'Jehoshapha', 'Jehovahnissi', 'Jehu', 'Jemuel', 'Jeremiah',
    'Jerusalem', 'Jesus', 'Jethro', 'Jews', 'Jezer', 'Jezreel', 'Jimnah',
    'Joash', 'Job', 'Jochebed', 'Joel', 'Johanan', 'John', 'Jonah', 'Jonas',
    'Joppa', 'Jordan', 'Josedech', 'Joseph', 'Josiah', 'Jotham', 'Judaea',
    'Judah', 'Judas', 'Julius', 'Just ', 'Justus ', 'Kadesh', 'Kareah',
    'Kedar', 'Kerioth', 'King', 'Kiriathaim ', 'Kohath', 'Korah', 'Korhites',
    'LORD', 'Laban', 'Laban', 'Lachis', 'Lamech', 'Laodicea', 'Lasea', 'Leah',
    'Lebanon', 'Legion', 'Levi', 'Libertines', 'Libni', 'Libyans', 'Loammi',
    'Lord', 'Lord Jesus Christ', 'Loruhamah', 'Lucas', 'Luhith', 'Luz', 'Lycia',
    'Lydians', 'Maaseiah', 'Macedonia', 'Macedonian', 'Machir', 'Machpelah',
    'Mahalaleel', 'Mahali', 'Malachi', 'Malchiel', 'Mamre', 'Manasseh',
    'Marcus', 'Mareshah', 'Maroth', 'Massah', 'Matthew', 'Matthias',
    'Melchizedek', 'Melita', 'Melzar', 'Merari', 'Meribah', 'Messias',
    'Methuselah', 'Micah', 'Midian', 'Migdol', 'Miriam', 'Misgab', 'Mishael',
    'Mizpah', 'Moab', 'Morasthite', 'Moreshethgath', 'Moses', 'Muppim', 'Mushi',
    'Myra', 'Naaman', 'Naashon', 'Nadab', 'Naphtali', 'Nathanael', 'Nazareth',
    'Nebo', 'Nebuchadnezzar', 'Nebuchadrezzar', 'Nebuzaradan', 'Nehelamite',
    'Nepheg', 'Neriah', 'Nicanor', 'Nicolas', 'Nicopolis', 'Nineveh', 'No ',
    'Noah', 'Noph', 'Ohad', 'Olivet', 'Omri', 'Onan', 'Onesimus', 'Padan',
    'Padanaram', 'Palestine', 'Pallu', 'Pamphylia', 'Parmenas', 'Pashur',
    'Pathros', 'Paul', 'Pentecost', 'Pergamos', 'Perizzite', 'Perizzites',
    'Peter', 'Pethuel', 'Phallu', 'Pharaoh', "Pharaoh's", 'Pharaohhophra ',
    'Pharaohnecho', 'Pharez', 'Pharisee ', 'Pharisees', 'Phenice',
    'Philadelphia', 'Philemon', 'Philip', 'Philippi', 'Philistines',
    'Phinehas', 'Phuvah', 'Pihahiroth', 'Pilate', 'Pithom', 'Pollux',
    'Potiphar', 'Potipherah', 'Prince', 'Priscilla', 'Prochorus', 'Puah',
    'Publius', 'Puteoli', 'Putiel', 'Raamses', 'Rachel', 'Ramah', 'Rameses',
    'Rephidim', 'Reuben', 'Reuel', 'Rhegium', 'Rock', 'Rome', 'Rosh', 'Ruhamah',
    'Sabeans', 'Sadducees', 'Salem', 'Salmone', 'Samaria', 'Samaritans',
    'Samuel', 'Saphir', 'Sara', 'Sarah', 'Sarai', 'Sardis', 'Satan', 'Saul',
    'Saviour', 'Scythian', 'Selah', 'Sepharad', 'Serah', 'Sered', 'Seth',
    'Shaphan', 'Shaul', 'Shelah', 'Shem', 'Shemaiah', 'Shillem', 'Shiloh',
    'Shimi', 'Shimron', 'Shiphrah', 'Shittim', 'Shuni', 'Sidon', 'Sihon',
    'Sihor', 'Simeon', 'Simon ', 'Simon Zelotes', 'Sinai', 'Sirion', 'Smyrna',
    'Solomon', 'Sosthenes', 'Stephanas', 'Stephen', 'Succoth', 'Sychar',
    'Syracuse', 'Tabor', 'Tahpanhes', 'Tarshish', 'Teman', 'Theophilus',
    'Thessalonica', 'Thomas', 'Thummim', 'Thyatira', 'Tiberias', 'Timon',
    'Timotheus', 'Timothy', 'Titus', 'Tobijah', 'Tola', 'Tophet', 'Tychicus',
    'Tyre', 'Tyrus', 'Ucal', 'Uri', 'Urim', 'Uzziah', 'Uzziel', 'Zaanan',
    'Zaphnathpaaneah', 'Zarephath', 'Zebedee', 'Zebulun', 'Zedekiah',
    'Zephaniah', 'Zerah', 'Zichri', 'Zidon', 'Zilpah', 'Zion', 'Ziphion',
    'Zipporah', 'Zithri', 'Zohar']

##########
def removePunc(input):
    no_punc = ''
    for char in input:
        if char not in punctuations:
            no_punc = no_punc + char
    return no_punc


##########
def generatedLatexFormattedText(dict):
    BUILDAWIP = 1
    BUILDNWIV = 1
    BUILDPNIP = 1
    BUILDNWIC = 1
    BUILDNLIW = 1
    BUILDBPIP = 1
    MINISTRY = 1

    topString = "\\footnote{\\textcolor[cmyk]{0.99998,1,0,0}{\\hyperlink{TOC}{Return to end " \
                "of Table of Contents.}}}"
    textColorString = "\\textcolor[cmyk]{0.99998,1,0,0}{"
    endString = "}"

    '''
    Print plain text with Latex color header:
        \textcolor[rgb]{0.00,0.00,1.00} == BLUE
        \textcolor[cmyk]{0.99998,1,0,0} == A DEEPER BLUE
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with Latex format\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')

    linesWith13Words = []
    linesWith18Words = []
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        newline = line[5]
        if versn2 == '1':
            print(topString + textColorString + newline + endString)
        else:
            print('[' + versn2 + '] ' + textColorString + newline + endString)
        words = newline.split()
        wordLength = len(words)
        if wordLength == 13:
            linesWith13Words.append(line)
        if wordLength == 18:
            linesWith18Words.append(line)
            
        
    '''
    NOW print the verses with LaTeX index items:
        AWIP - All Words in Passage
        NWIV - NUmber of Words in Verse
        PNIP - Proper noun in passage
        ... more later
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with indexed items\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        verseRef = "!" + booknm + "!" + chapnm + " " + chapno + ":" + versno 
        line = line[5]
        if versn2 == '1':
            verseString = topString + textColorString + line + endString
        else:
            verseString = '[' + versn2 + '] ' + textColorString + line + endString
        #print(booknm,chapnm,chapno,versno,versn2) 
        '''
        NWIV
        '''
        words = line.split()
        numWords = len(words)
        if DEBUG == 1:
            print(numWords,'Words in verse',versn2)
        extraString="\\index[NWIV]{" + str(numWords) + verseRef + "}"
        '''
        AWIP - here, Passage = verse
        '''
        verseDict = {}
        wordString = ''
        for word in words:
            word = removePunc(word)
            if word not in verseDict:
                verseDict[word] = 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef + "}"
            else:
                verseDict[word] = verseDict[word] + 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef 
                wordString = wordString + " (" + str(verseDict[word]) + ")}"
        '''
        PNIP - Proper names in passage
        '''
        propernameStrng = ''
        for item in verseDict:
            if item in properNameList:
                if item not in exceptions:
                    propernameStrng = propernameStrng + "\\index[PNIP]{" + item + verseRef + "}"

        builtVerseString = verseString + extraString + wordString  + propernameStrng
        print(builtVerseString)
    return linesWith18Words, linesWith13Words

##########
def showWordsByFrequency(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words by Frequency}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[1], reverse=True)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def showWordsAlphabetically(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words Alphabetically}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[0], reverse=False)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def generateWordStatisticsForChapter(label,verses):
    '''
    Need to simplify this code! ONLY count number of words, number of unique 
    words, number of itallic words, number of unique itallic words
    
    verseDictionary, rest for each verse
    chapterDictionary
    '''
    global properNameList
    wordCount = 0
    emphasizedwordCount = 0
    wordDict = {}
    emphasizedwordDict = {}
    verseString = ''
    A = re.findall(r'\d+', label)
    A = " ".join(map(str, A))
    digit = int(A)

    word1 = " ".join(re.findall("[a-zA-Z]+", label))
    
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('%%%%% Word Statistics\n%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    
    print('\\normalsize')

    verseStats = ["\\subsection{Chapter Word Statistics}",
                  "\n",
                  "%%%%%%%%%%",
                  "%%%%%%%%%%",
                  " ",
                  "\\begin{center}",
                  "\\begin{longtable}{l|c|c|c|c}",
                  "\\caption[Stats for "+word1+" "+str(digit) +"]{Stats for "+word1+" "+str(digit) +"} \\label{table:Stats for "+word1+" "+str(digit) +"} \\\ ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endfirsthead",
                  " ",
                  "\multicolumn{5}{c}",
                  "{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\  ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endhead",
                  " ",
                  "\\hline \\multicolumn{5}{|r|}{{Continued if needed}} \\\ \\hline",
                  "\\endfoot ",]

    '''
    This section prints JUST the Bible text in a LaTeX format with a textcolor encoding added
    Original Code
    '''
    chapterDict = {}
    chapwordWords = 0
    chapterUniqueWords = 0
    numberChapterUniqueItallics = 0
    numberChapterItallics = 0
    verseNo = 0
    for line in verses:
        verseNo += 1
        verseDict = {}
#        print(line)
        words = removePunc(line[5])
        wordcount = 0
        uniqueWords = 0        
        '''
        Process for verse
        '''
        for word in words.split():
            wordcount += 1
            if word not in verseDict:
                verseDict[word] = 1
                uniqueWords += 1
            else:
                verseDict[word] = verseDict[word] + 1
        numberUniqueItallics = 0
        numberItallics = 0
        for item in verseDict:
            if "\emph{" in item:
                numberUniqueItallics += 1
                numberItallics = numberItallics + verseDict[item]                
        '''
        Process for chapter
        '''                
        for word in words.split():
            chapwordWords += 1
            if word not in chapterDict:
                chapterDict[word] = 1
                chapterUniqueWords += 1
            else:
                chapterDict[word] = chapterDict[word] + 1
        sequence = str(verseNo),str(wordcount),str(uniqueWords),str(numberItallics),str(numberUniqueItallics)+'\\\ \hline'
        S = ' & '.join(sequence)
        verseStats.append(S)
    for item in chapterDict:
        if "\emph{" in item:
            numberChapterUniqueItallics +=1
            numberChapterItallics = numberChapterItallics + chapterDict[item]

    '''
    Finish Up!
    '''
#    print ("\n",wordCount,"words in passage, of which",len(wordDict),"are unique")
#    print (emphasizedwordCount,'words in ital, of which',len(emphasizedwordDict),'are unique')
    print ("\n\n")
    verseStats.append("\hline \hline")
    sequence = 'Total',str(chapwordWords),str(chapterUniqueWords),str(numberChapterItallics),str(numberChapterUniqueItallics)
    S = ' & '.join(sequence)
    verseStats.append(S)
    for item in verseStats:
        print (item)
    print ("\n\n")
    print ("\\end{longtable}")
    print ("\\end{center}\n")
#    print (chapwordWords,"words in passage, of which",chapterUniqueWords,"are unique")
#    print (numberChapterItallics,'words in ital, of which',numberChapterUniqueItallics,'are unique')
#    print ("\n\n")
#
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
    
    return chapterDict

##########
def getWordLengthsInChapter(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    wordLengthDictionary = {}
    for item in dictionary:
        if "\emph{" in item:
            tempitem = item.replace("\emph{",'')
            tempitem = tempitem.replace('}','')
            if len(tempitem) in wordLengthDictionary:
                wordLengthDictionary[len(tempitem)] = wordLengthDictionary[len(tempitem)] + ', ' + item
            else:
                    wordLengthDictionary[len(tempitem)] = item
        else:
            if len(item) in wordLengthDictionary:
                wordLengthDictionary[len(item)] = wordLengthDictionary[len(item)] + ', ' + item
            else:
                wordLengthDictionary[len(item)] = item
    wordLengthDictionary = sorted(wordLengthDictionary.items(), key=lambda x:x[0], reverse=False)

    print('\\subsection{Word Lengths in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"]{Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in wordLengthDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")

##########
def makeVersesWith13WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 13 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 13 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########
          
##########
def makeVersesWith18WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 18 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 18 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########

def countLettersInVerses(arg1,arg2):
    for item in arg2:
        letterDict = {}
        for item2 in item[5]:
            if "\emph" in item2:
                item2 = item2.replace('\emph','')
            if item2 != ' ':
                if item2 in letterDict:
                    letterDict[item2] = letterDict[item2] + 1
                else:
                    letterDict[item2] =  1
            print ("for verse",item[2],":",letterDict)

def ProcessVerses(arg1,arg2):
    ''' 
    arg1 is dictionary name and arg2 is the dictionary
    '''
#    saveToPickleFile(arg1,arg2)  
    linesWith18words, linesWith13words = generatedLatexFormattedText(arg2)         # format chapter in Latex
    #linesWith13words is for use later ... 
    chapterDict = generateWordStatisticsForChapter(arg1,arg2)   # do chapter word statistics
    showWordsByFrequency(chapterDict,arg1)                      # show words by frequency
    showWordsAlphabetically(chapterDict,arg1)                   # show words alphabetically
    getWordLengthsInChapter(chapterDict,arg1)                   # show word lengths in chapter
    
    if len(linesWith13words) > 0:
        #print(linesWith13words)
        makeVersesWith13WordsTable(linesWith13words,arg1)
        
    if len(linesWith18words) > 0:
        #print(linesWith13words)
        makeVersesWith18WordsTable(linesWith18words,arg1)
        
    ''' In Development
    countLettersInVerses(arg1,arg2)
    '''



In [3]:
path = 'F:\python'
os.chdir(path)
print (os.getcwd())

F:\python


In [ ]:
# %load NEW18OT-MakeConcordance-Job.py
"""
Created on Sun Jan 01 07:52:18 2017

@author: keith
"""
import string

from BibleWordList2 import *

Job01 = [
['Job','Job','01','001','1',"There was a man in the land of Uz, whose name \emph{was} Job; and that man was perfect and upright, and one that feared God, and eschewed evil."],
['Job','Job','01','002','2',"And there were born unto him seven sons and three daughters."],
['Job','Job','01','003','3',"His substance also was seven thousand sheep, and three thousand camels, and five hundred yoke of oxen, and five hundred she asses, and a very great household; so that this man was the greatest of all the men of the east."],
['Job','Job','01','004','4',"And his sons went and feasted \emph{in} \emph{their} houses, every one his day; and sent and called for their three sisters to eat and to drink with them."],
['Job','Job','01','005','5',"And it was so, when the days of \emph{their} feasting were gone about, that Job sent and sanctified them, and rose up early in the morning, and offered burnt offerings \emph{according} to the number of them all: for Job said, It may be that my sons have sinned, and cursed God in their hearts. Thus did Job continually."],
['Job','Job','01','006','6',"Now there was a day when the sons of God came to present themselves before the LORD, and Satan came also among them."],
['Job','Job','01','007','7',"And the LORD said unto Satan, Whence comest thou? Then Satan answered the LORD, and said, From going to and fro in the earth, and from walking up and down in it."],
['Job','Job','01','008','8',"And the LORD said unto Satan, Hast thou considered my servant Job, that \emph{there} \emph{is} none like him in the earth, a perfect and an upright man, one that feareth God, and escheweth evil?"],
['Job','Job','01','009','9',"Then Satan answered the LORD, and said, Doth Job fear God for nought?"],
['Job','Job','01','010','10',"Hast not thou made an hedge about him, and about his house, and about all that he hath on every side? thou hast blessed the work of his hands, and his substance is increased in the land."],
['Job','Job','01','011','11',"But put forth thine hand now, and touch all that he hath, and he will curse thee to thy face."],
['Job','Job','01','012','12',"And the LORD said unto Satan, Behold, all that he hath \emph{is} in thy power; only upon himself put not forth thine hand. So Satan went forth from the presence of the LORD."],
['Job','Job','01','013','13',"And there was a day when his sons and his daughters \emph{were} eating and drinking wine in their eldest brother's house:"],
['Job','Job','01','014','14',"And there came a messenger unto Job, and said, The oxen were plowing, and the asses feeding beside them:"],
['Job','Job','01','015','15',"And the Sabeans fell \emph{upon} \emph{them}, and took them away; yea, they have slain the servants with the edge of the sword; and I only am escaped alone to tell thee."],
['Job','Job','01','016','16',"While he \emph{was} yet speaking, there came also another, and said, The fire of God is fallen from heaven, and hath burned up the sheep, and the servants, and consumed them; and I only am escaped alone to tell thee."],
['Job','Job','01','017','17',"While he \emph{was} yet speaking, there came also another, and said, The Chaldeans made out three bands, and fell upon the camels, and have carried them away, yea, and slain the servants with the edge of the sword; and I only am escaped alone to tell thee."],
['Job','Job','01','018','18',"While he \emph{was} yet speaking, there came also another, and said, Thy sons and thy daughters \emph{were} eating and drinking wine in their eldest brother's house:"],
['Job','Job','01','019','19',"And, behold, there came a great wind from the wilderness, and smote the four corners of the house, and it fell upon the young men, and they are dead; and I only am escaped alone to tell thee."],
['Job','Job','01','020','20',"Then Job arose, and rent his mantle, and shaved his head, and fell down upon the ground, and worshipped,"],
['Job','Job','01','021','21',"And said, Naked came I out of my mother's womb, and naked shall I return thither: the LORD gave, and the LORD hath taken away; blessed be the name of the LORD."],
['Job','Job','01','022','22',"In all this Job sinned not, nor charged God foolishly."]
]

Job02 = [
    ['Job','Job','02','001','1',"Again there was a day when the sons of God came to present themselves before the LORD, and Satan came also among them to present himself before the LORD."],
    ['Job','Job','02','002','2',"And the LORD said unto Satan, From whence comest thou? And Satan answered the LORD, and said, From going to and fro in the earth, and from walking up and down in it."],
    ['Job','Job','02','003','3',"And the LORD said unto Satan, Hast thou considered my servant Job, that \emph{there} \emph{is} none like him in the earth, a perfect and an upright man, one that feareth God, and escheweth evil? and still he holdeth fast his integrity, although thou movedst me against him, to destroy him without cause."],
    ['Job','Job','02','004','4',"And Satan answered the LORD, and said, Skin for skin, yea, all that a man hath will he give for his life."],
    ['Job','Job','02','005','5',"But put forth thine hand now, and touch his bone and his flesh, and he will curse thee to thy face."],
    ['Job','Job','02','006','6',"And the LORD said unto Satan, Behold, he \emph{is} in thine hand; but save his life."],
    ['Job','Job','02','007','7',"So went Satan forth from the presence of the LORD, and smote Job with sore boils from the sole of his foot unto his crown."],
    ['Job','Job','02','008','8',"And he took him a potsherd to scrape himself withal; and he sat down among the ashes."],
    ['Job','Job','02','009','9',"Then said his wife unto him, Dost thou still retain thine integrity? curse God, and die."],
    ['Job','Job','02','010','10',"But he said unto her, Thou speakest as one of the foolish women speaketh. What? shall we receive good at the hand of God, and shall we not receive evil? In all this did not Job sin with his lips."],
    ['Job','Job','02','011','11',"Now when Job's three friends heard of all this evil that was come upon him, they came every one from his own place; Eliphaz the Temanite, and Bildad the Shuhite, and Zophar the Naamathite: for they had made an appointment together to come to mourn with him and to comfort him."],
    ['Job','Job','02','012','12',"And when they lifted up their eyes afar off, and knew him not, they lifted up their voice, and wept; and they rent every one his mantle, and sprinkled dust upon their heads toward heaven."],
    ['Job','Job','02','013','13',"So they sat down with him upon the ground seven days and seven nights, and none spake a word unto him: for they saw that \emph{his} grief was very great."],
]

Job03 = [
    ['Job','Job','03','001','1',"After this opened Job his mouth, and cursed his day."],
    ['Job','Job','03','002','2',"And Job spake, and said,"],
    ['Job','Job','03','003','3',"Let the day perish wherein I was born, and the night \emph{in} \emph{which} it was said, There is a man child conceived."],
    ['Job','Job','03','004','4',"Let that day be darkness; let not God regard it from above, neither let the light shine upon it."],
    ['Job','Job','03','005','5',"Let darkness and the shadow of death stain it; let a cloud dwell upon it; let the blackness of the day terrify it."],
    ['Job','Job','03','006','6',"As \emph{for} that night, let darkness seize upon it; let it not be joined unto the days of the year, let it not come into the number of the months."],
    ['Job','Job','03','007','7',"Lo, let that night be solitary, let no joyful voice come therein."],
    ['Job','Job','03','008','8',"Let them curse it that curse the day, who are ready to raise up their mourning."],
    ['Job','Job','03','009','9',"Let the stars of the twilight thereof be dark; let it look for light, but \emph{have} none; neither let it see the dawning of the day:"],
    ['Job','Job','03','010','10',"Because it shut not up the doors of my \emph{mother's} womb, nor hid sorrow from mine eyes."],
    ['Job','Job','03','011','11',"Why died I not from the womb? \emph{why} did I \emph{not} give up the ghost when I came out of the belly?"],
    ['Job','Job','03','012','12',"Why did the knees prevent me? or why the breasts that I should suck?"],
    ['Job','Job','03','013','13',"For now should I have lain still and been quiet, I should have slept: then had I been at rest,"],
    ['Job','Job','03','014','14',"With kings and counsellors of the earth, which built desolate places for themselves;"],
    ['Job','Job','03','015','15',"Or with princes that had gold, who filled their houses with silver:"],
    ['Job','Job','03','016','16',"Or as an hidden untimely birth I had not been; as infants \emph{which} never saw light."],
    ['Job','Job','03','017','17',"There the wicked cease \emph{from} troubling; and there the weary be at rest."],
    ['Job','Job','03','018','18',"\emph{There} the prisoners rest together; they hear not the voice of the oppressor."],
    ['Job','Job','03','019','19',"The small and great are there; and the servant \emph{is} free from his master."],
    ['Job','Job','03','020','20',"Wherefore is light given to him that is in misery, and life unto the bitter \emph{in} soul;"],
    ['Job','Job','03','021','21',"Which long for death, but it \emph{cometh} not; and dig for it more than for hid treasures;"],
    ['Job','Job','03','022','22',"Which rejoice exceedingly, \emph{and} are glad, when they can find the grave?"],
    ['Job','Job','03','023','23',"\emph{Why} \emph{is} \emph{light} \emph{given} to a man whose way is hid, and whom God hath hedged in?"],
    ['Job','Job','03','024','24',"For my sighing cometh before I eat, and my roarings are poured out like the waters."],
    ['Job','Job','03','025','25',"For the thing which I greatly feared is come upon me, and that which I was afraid of is come unto me."],
    ['Job','Job','03','026','26',"I was not in safety, neither had I rest, neither was I quiet; yet trouble came."]
]


Job04 = [
['Job','Job','04','001','1',"Then Eliphaz the Temanite answered and said,"],
['Job','Job','04','002','2',"\emph{If} we assay to commune with thee, wilt thou be grieved? but who can withhold himself from speaking?"],
['Job','Job','04','003','3',"Behold, thou hast instructed many, and thou hast strengthened the weak hands."],
['Job','Job','04','004','4',"Thy words have upholden him that was falling, and thou hast strengthened the feeble knees."],
['Job','Job','04','005','5',"But now it is come upon thee, and thou faintest; it toucheth thee, and thou art troubled."],
['Job','Job','04','006','6',"\emph{Is} not \emph{this} thy fear, thy confidence, thy hope, and the uprightness of thy ways?"],
['Job','Job','04','007','7',"Remember, I pray thee, who \emph{ever} perished, being innocent? or where were the righteous cut off?"],
['Job','Job','04','008','8',"Even as I have seen, they that plow iniquity, and sow wickedness, reap the same."],
['Job','Job','04','009','9',"By the blast of God they perish, and by the breath of his nostrils are they consumed."],
['Job','Job','04','010','10',"The roaring of the lion, and the voice of the fierce lion, and the teeth of the young lions, are broken."],
['Job','Job','04','011','11',"The old lion perisheth for lack of prey, and the stout lion's whelps are scattered abroad."],
['Job','Job','04','012','12',"Now a thing was secretly brought to me, and mine ear received a little thereof."],
['Job','Job','04','013','13',"In thoughts from the visions of the night, when deep sleep falleth on men,"],
['Job','Job','04','014','14',"Fear came upon me, and trembling, which made all my bones to shake."],
['Job','Job','04','015','15',"Then a spirit passed before my face; the hair of my flesh stood up:"],
['Job','Job','04','016','16',"It stood still, but I could not discern the form thereof: an image \emph{was} before mine eyes, \emph{there} \emph{was} silence, and I heard a voice, \emph{saying},"],
['Job','Job','04','017','17',"Shall mortal man be more just than God? shall a man be more pure than his maker?"],
['Job','Job','04','018','18',"Behold, he put no trust in his servants; and his angels he charged with folly:"],
['Job','Job','04','019','19',"How much less \emph{in} them that dwell in houses of clay, whose foundation \emph{is} in the dust, \emph{which} are crushed before the moth?"],
['Job','Job','04','020','20',"They are destroyed from morning to evening: they perish for ever without any regarding \emph{it}."],
['Job','Job','04','021','21',"Doth not their excellency \emph{which} \emph{is} in them go away? they die, even without wisdom."]
]

Job05 = [
['Job','Job','05','001','1',"Call now, if there be any that will answer thee; and to which of the saints wilt thou turn?"],
['Job','Job','05','002','2',"For wrath killeth the foolish man, and envy slayeth the silly one."],
['Job','Job','05','003','3',"I have seen the foolish taking root: but suddenly I cursed his habitation."],
['Job','Job','05','004','4',"His children are far from safety, and they are crushed in the gate, neither \emph{is} \emph{there} any to deliver \emph{them}."],
['Job','Job','05','005','5',"Whose harvest the hungry eateth up, and taketh it even out of the thorns, and the robber swalloweth up their substance."],
['Job','Job','05','006','6',"Although affliction cometh not forth of the dust, neither doth trouble spring out of the ground;"],
['Job','Job','05','007','7',"Yet man is born unto trouble, as the sparks fly upward."],
['Job','Job','05','008','8',"I would seek unto God, and unto God would I commit my cause:"],
['Job','Job','05','009','9',"Which doeth great things and unsearchable; marvellous things without number:"],
['Job','Job','05','010','10',"Who giveth rain upon the earth, and sendeth waters upon the fields:"],
['Job','Job','05','011','11',"To set up on high those that be low; that those which mourn may be exalted to safety."],
['Job','Job','05','012','12',"He disappointeth the devices of the crafty, so that their hands cannot perform \emph{their} enterprise."],
['Job','Job','05','013','13',"He taketh the wise in their own craftiness: and the counsel of the froward is carried headlong."],
['Job','Job','05','014','14',"They meet with darkness in the daytime, and grope in the noonday as in the night."],
['Job','Job','05','015','15',"But he saveth the poor from the sword, from their mouth, and from the hand of the mighty."],
['Job','Job','05','016','16',"So the poor hath hope, and iniquity stoppeth her mouth."],
['Job','Job','05','017','17',"Behold, happy \emph{is} the man whom God correcteth: therefore despise not thou the chastening of the Almighty:"],
['Job','Job','05','018','18',"For he maketh sore, and bindeth up: he woundeth, and his hands make whole."],
['Job','Job','05','019','19',"He shall deliver thee in six troubles: yea, in seven there shall no evil touch thee."],
['Job','Job','05','020','20',"In famine he shall redeem thee from death: and in war from the power of the sword."],
['Job','Job','05','021','21',"Thou shalt be hid from the scourge of the tongue: neither shalt thou be afraid of destruction when it cometh."],
['Job','Job','05','022','22',"At destruction and famine thou shalt laugh: neither shalt thou be afraid of the beasts of the earth."],
['Job','Job','05','023','23',"For thou shalt be in league with the stones of the field: and the beasts of the field shall be at peace with thee."],
['Job','Job','05','024','24',"And thou shalt know that thy tabernacle \emph{shall} \emph{be} in peace; and thou shalt visit thy habitation, and shalt not sin."],
['Job','Job','05','025','25',"Thou shalt know also that thy seed \emph{shall} \emph{be} great, and thine offspring as the grass of the earth."],
['Job','Job','05','026','26',"Thou shalt come to \emph{thy} grave in a full age, like as a shock of corn cometh in in his season."],
['Job','Job','05','027','27',"Lo this, we have searched it, so it \emph{is}; hear it, and know thou \emph{it} for thy good."]
]

Job06 = [
['Job','Job','06','001','1',"But Job answered and said,"],
['Job','Job','06','002','2',"Oh that my grief were throughly weighed, and my calamity laid in the balances together!"],
['Job','Job','06','003','3',"For now it would be heavier than the sand of the sea: therefore my words are swallowed up."],
['Job','Job','06','004','4',"For the arrows of the Almighty \emph{are} within me, the poison whereof drinketh up my spirit: the terrors of God do set themselves in array against me."],
['Job','Job','06','005','5',"Doth the wild ass bray when he hath grass? or loweth the ox over his fodder?"],
['Job','Job','06','006','6',"Can that which is unsavoury be eaten without salt? or is there \emph{any} taste in the white of an egg?"],
['Job','Job','06','007','7',"The things \emph{that} my soul refused to touch \emph{are} as my sorrowful meat."],
['Job','Job','06','008','8',"Oh that I might have my request; and that God would grant \emph{me} the thing that I long for!"],
['Job','Job','06','009','9',"Even that it would please God to destroy me; that he would let loose his hand, and cut me off!"],
['Job','Job','06','010','10',"Then should I yet have comfort; yea, I would harden myself in sorrow: let him not spare; for I have not concealed the words of the Holy One."],
['Job','Job','06','011','11',"What \emph{is} my strength, that I should hope? and what \emph{is} mine end, that I should prolong my life?"],
['Job','Job','06','012','12',"\emph{Is} my strength the strength of stones? or \emph{is} my flesh of brass?"],
['Job','Job','06','013','13',"\emph{Is} not my help in me? and is wisdom driven quite from me?"],
['Job','Job','06','014','14',"To him that is afflicted pity \emph{should} \emph{be} \emph{shewed} from his friend; but he forsaketh the fear of the Almighty."],
['Job','Job','06','015','15',"My brethren have dealt deceitfully as a brook, \emph{and} as the stream of brooks they pass away;"],
['Job','Job','06','016','16',"Which are blackish by reason of the ice, \emph{and} wherein the snow is hid:"],
['Job','Job','06','017','17',"What time they wax warm, they vanish: when it is hot, they are consumed out of their place."],
['Job','Job','06','018','18',"The paths of their way are turned aside; they go to nothing, and perish."],
['Job','Job','06','019','19',"The troops of Tema looked, the companies of Sheba waited for them."],
['Job','Job','06','020','20',"They were confounded because they had hoped; they came thither, and were ashamed."],
['Job','Job','06','021','21',"For now ye are nothing; ye see \emph{my} casting down, and are afraid."],
['Job','Job','06','022','22',"Did I say, Bring unto me? or, Give a reward for me of your substance?"],
['Job','Job','06','023','23',"Or, Deliver me from the enemy's hand? or, Redeem me from the hand of the mighty?"],
['Job','Job','06','024','24',"Teach me, and I will hold my tongue: and cause me to understand wherein I have erred."],
['Job','Job','06','025','25',"How forcible are right words! but what doth your arguing reprove?"],
['Job','Job','06','026','26',"Do ye imagine to reprove words, and the speeches of one that is desperate, \emph{which} \emph{are} as wind?"],
['Job','Job','06','027','27',"Yea, ye overwhelm the fatherless, and ye dig \emph{a} \emph{pit} for your friend."],
['Job','Job','06','028','28',"Now therefore be content, look upon me; for \emph{it} \emph{is} evident unto you if I lie."],
['Job','Job','06','029','29',"Return, I pray you, let it not be iniquity; yea, return again, my righteousness \emph{is} in it."],
['Job','Job','06','030','30',"Is there iniquity in my tongue? cannot my taste discern perverse things?"]
]

Job07 = [
    ['Job','Job','07','001','1',"\emph{Is} \emph{there} not an appointed time to man upon earth? \emph{are} \emph{not} his days also like the days of an hireling?"],
    ['Job','Job','07','002','2',"As a servant earnestly desireth the shadow, and as an hireling looketh for \emph{the} \emph{reward} \emph{of} his work:"],
    ['Job','Job','07','003','3',"So am I made to possess months of vanity, and wearisome nights are appointed to me."],
    ['Job','Job','07','004','4',"When I lie down, I say, When shall I arise, and the night be gone? and I am full of tossings to and fro unto the dawning of the day."],
    ['Job','Job','07','005','5',"My flesh is clothed with worms and clods of dust; my skin is broken, and become loathsome."],
    ['Job','Job','07','006','6',"My days are swifter than a weaver's shuttle, and are spent without hope."],
    ['Job','Job','07','007','7',"O remember that my life \emph{is} wind: mine eye shall no more see good."],
    ['Job','Job','07','008','8',"The eye of him that hath seen me shall see me no \emph{more}: thine eyes \emph{are} upon me, and I \emph{am} not."],
    ['Job','Job','07','009','9',"\emph{As} the cloud is consumed and vanisheth away: so he that goeth down to the grave shall come up no \emph{more}."],
    ['Job','Job','07','010','10',"He shall return no more to his house, neither shall his place know him any more."],
    ['Job','Job','07','011','11',"Therefore I will not refrain my mouth; I will speak in the anguish of my spirit; I will complain in the bitterness of my soul."],
    ['Job','Job','07','012','12',"\emph{Am} I a sea, or a whale, that thou settest a watch over me?"],
    ['Job','Job','07','013','13',"When I say, My bed shall comfort me, my couch shall ease my complaint;"],
    ['Job','Job','07','014','14',"Then thou scarest me with dreams, and terrifiest me through visions:"],
    ['Job','Job','07','015','15',"So that my soul chooseth strangling, \emph{and} death rather than my life."],
    ['Job','Job','07','016','16',"I loathe \emph{it}; I would not live alway: let me alone; for my days \emph{are} vanity."],
    ['Job','Job','07','017','17',"What \emph{is} man, that thou shouldest magnify him? and that thou shouldest set thine heart upon him?"],
    ['Job','Job','07','018','18',"And \emph{that} thou shouldest visit him every morning, \emph{and} try him every moment?"],
    ['Job','Job','07','019','19',"How long wilt thou not depart from me, nor let me alone till I swallow down my spittle?"],
    ['Job','Job','07','020','20',"I have sinned; what shall I do unto thee, O thou preserver of men? why hast thou set me as a mark against thee, so that I am a burden to myself?"],
    ['Job','Job','07','021','21',"And why dost thou not pardon my transgression, and take away mine iniquity? for now shall I sleep in the dust; and thou shalt seek me in the morning, but I \emph{shall} not \emph{be}."]
]

Job08 = [
['Job','Job','08','001','1',"Then answered Bildad the Shuhite, and said,"],
['Job','Job','08','002','2',"How long wilt thou speak these \emph{things}? and \emph{how} \emph{long} \emph{shall} the words of thy mouth \emph{be} \emph{like} a strong wind?"],
['Job','Job','08','003','3',"Doth God pervert judgment? or doth the Almighty pervert justice?"],
['Job','Job','08','004','4',"If thy children have sinned against him, and he have cast them away for their transgression;"],
['Job','Job','08','005','5',"If thou wouldest seek unto God betimes, and make thy supplication to the Almighty;"],
['Job','Job','08','006','6',"If thou \emph{wert} pure and upright; surely now he would awake for thee, and make the habitation of thy righteousness prosperous."],
['Job','Job','08','007','7',"Though thy beginning was small, yet thy latter end should greatly increase."],
['Job','Job','08','008','8',"For enquire, I pray thee, of the former age, and prepare thyself to the search of their fathers:"],
['Job','Job','08','009','9',"(For we \emph{are} \emph{but} \emph{of} yesterday, and know nothing, because our days upon earth \emph{are} a shadow:)"],
['Job','Job','08','010','10',"Shall not they teach thee, \emph{and} tell thee, and utter words out of their heart?"],
['Job','Job','08','011','11',"Can the rush grow up without mire? can the flag grow without water?"],
['Job','Job','08','012','12',"Whilst it \emph{is} yet in his greenness, \emph{and} not cut down, it withereth before any \emph{other} herb."],
['Job','Job','08','013','13',"So \emph{are} the paths of all that forget God; and the hypocrite's hope shall perish:"],
['Job','Job','08','014','14',"Whose hope shall be cut off, and whose trust \emph{shall} \emph{be} a spider's web."],
['Job','Job','08','015','15',"He shall lean upon his house, but it shall not stand: he shall hold it fast, but it shall not endure."],
['Job','Job','08','016','16',"He \emph{is} green before the sun, and his branch shooteth forth in his garden."],
['Job','Job','08','017','17',"His roots are wrapped about the heap, \emph{and} seeth the place of stones."],
['Job','Job','08','018','18',"If he destroy him from his place, then \emph{it} shall deny him, \emph{saying}, I have not seen thee."],
['Job','Job','08','019','19',"Behold, this \emph{is} the joy of his way, and out of the earth shall others grow."],
['Job','Job','08','020','20',"Behold, God will not cast away a perfect \emph{man}, neither will he help the evil doers:"],
['Job','Job','08','021','21',"Till he fill thy mouth with laughing, and thy lips with rejoicing."],
['Job','Job','08','022','22',"They that hate thee shall be clothed with shame; and the dwelling place of the wicked shall come to nought."]
]

Job09 = [
    ['Job','Job','09','001','1',"Then Job answered and said,"],
    ['Job','Job','09','002','2',"I know \emph{it} \emph{is} so of a truth: but how should man be just with God?"],
    ['Job','Job','09','003','3',"If he will contend with him, he cannot answer him one of a thousand."],
    ['Job','Job','09','004','4',"\emph{He} \emph{is} wise in heart, and mighty in strength: who hath hardened \emph{himself} against him, and hath prospered?"],
    ['Job','Job','09','005','5',"Which removeth the mountains, and they know not: which overturneth them in his anger."],
    ['Job','Job','09','006','6',"Which shaketh the earth out of her place, and the pillars thereof tremble."],
    ['Job','Job','09','007','7',"Which commandeth the sun, and it riseth not; and sealeth up the stars."],
    ['Job','Job','09','008','8',"Which alone spreadeth out the heavens, and treadeth upon the waves of the sea."],
    ['Job','Job','09','009','9',"Which maketh Arcturus, Orion, and Pleiades, and the chambers of the south."],
    ['Job','Job','09','010','10',"Which doeth great things past finding out; yea, and wonders without number."],
    ['Job','Job','09','011','11',"Lo, he goeth by me, and I see \emph{him} not: he passeth on also, but I perceive him not."],
    ['Job','Job','09','012','12',"Behold, he taketh away, who can hinder him? who will say unto him, What doest thou?"],
    ['Job','Job','09','013','13',"\emph{If} God will not withdraw his anger, the proud helpers do stoop under him."],
    ['Job','Job','09','014','14',"How much less shall I answer him, \emph{and} choose out my words \emph{to} \emph{reason} with him?"],
    ['Job','Job','09','015','15',"Whom, though I were righteous, \emph{yet} would I not answer, \emph{but} I would make supplication to my judge."],
    ['Job','Job','09','016','16',"If I had called, and he had answered me; \emph{yet} would I not believe that he had hearkened unto my voice."],
    ['Job','Job','09','017','17',"For he breaketh me with a tempest, and multiplieth my wounds without cause."],
    ['Job','Job','09','018','18',"He will not suffer me to take my breath, but filleth me with bitterness."],
    ['Job','Job','09','019','19',"If \emph{I} \emph{speak} of strength, lo, \emph{he} \emph{is} strong: and if of judgment, who shall set me a time \emph{to} \emph{plead}?"],
    ['Job','Job','09','020','20',"If I justify myself, mine own mouth shall condemn me: \emph{if} \emph{I} \emph{say}, I \emph{am} perfect, it shall also prove me perverse."],
    ['Job','Job','09','021','21',"\emph{Though} I \emph{were} perfect, \emph{yet} would I not know my soul: I would despise my life."],
    ['Job','Job','09','022','22',"This \emph{is} one \emph{thing}, therefore I said \emph{it}, He destroyeth the perfect and the wicked."],
    ['Job','Job','09','023','23',"If the scourge slay suddenly, he will laugh at the trial of the innocent."],
    ['Job','Job','09','024','24',"The earth is given into the hand of the wicked: he covereth the faces of the judges thereof; if not, where, \emph{and} who \emph{is} he?"],
    ['Job','Job','09','025','25',"Now my days are swifter than a post: they flee away, they see no good."],
    ['Job','Job','09','026','26',"They are passed away as the swift ships: as the eagle \emph{that} hasteth to the prey."],
    ['Job','Job','09','027','27',"If I say, I will forget my complaint, I will leave off my heaviness, and comfort \emph{myself}:"],
    ['Job','Job','09','028','28',"I am afraid of all my sorrows, I know that thou wilt not hold me innocent."],
    ['Job','Job','09','029','29',"\emph{If} I be wicked, why then labour I in vain?"],
    ['Job','Job','09','030','30',"If I wash myself with snow water, and make my hands never so clean;"],
    ['Job','Job','09','031','31',"Yet shalt thou plunge me in the ditch, and mine own clothes shall abhor me."],
    ['Job','Job','09','032','32',"For \emph{he} \emph{is} not a man, as I \emph{am,} \emph{that} I should answer him, \emph{and} we should come together in judgment."],
    ['Job','Job','09','033','33',"Neither is there any daysman betwixt us, \emph{that} might lay his hand upon us both."],
    ['Job','Job','09','034','34',"Let him take his rod away from me, and let not his fear terrify me:"],
    ['Job','Job','09','035','35',"\emph{Then} would I speak, and not fear him; but \emph{it} \emph{is} not so with me."]
]

Job10 = [
    ['Job','Job','10','001','1',"My soul is weary of my life; I will leave my complaint upon myself; I will speak in the bitterness of my soul."],
    ['Job','Job','10','002','2',"I will say unto God, Do not condemn me; shew me wherefore thou contendest with me."],
    ['Job','Job','10','003','3',"\emph{Is} \emph{it} good unto thee that thou shouldest oppress, that thou shouldest despise the work of thine hands, and shine upon the counsel of the wicked?"],
    ['Job','Job','10','004','4',"Hast thou eyes of flesh? or seest thou as man seeth?"],
    ['Job','Job','10','005','5',"\emph{Are} thy days as the days of man? \emph{are} thy years as man's days,"],
    ['Job','Job','10','006','6',"That thou enquirest after mine iniquity, and searchest after my sin?"],
    ['Job','Job','10','007','7',"Thou knowest that I am not wicked; and \emph{there} \emph{is} none that can deliver out of thine hand."],
    ['Job','Job','10','008','8',"Thine hands have made me and fashioned me together round about; yet thou dost destroy me."],
    ['Job','Job','10','009','9',"Remember, I beseech thee, that thou hast made me as the clay; and wilt thou bring me into dust again?"],
    ['Job','Job','10','010','10',"Hast thou not poured me out as milk, and curdled me like cheese?"],
    ['Job','Job','10','011','11',"Thou hast clothed me with skin and flesh, and hast fenced me with bones and sinews."],
    ['Job','Job','10','012','12',"Thou hast granted me life and favour, and thy visitation hath preserved my spirit."],
    ['Job','Job','10','013','13',"And these \emph{things} hast thou hid in thine heart: I know that this \emph{is} with thee."],
    ['Job','Job','10','014','14',"If I sin, then thou markest me, and thou wilt not acquit me from mine iniquity."],
    ['Job','Job','10','015','15',"If I be wicked, woe unto me; and \emph{if} I be righteous, \emph{yet} will I not lift up my head. \emph{I} \emph{am} full of confusion; therefore see thou mine affliction;"],
    ['Job','Job','10','016','16',"For it increaseth. Thou huntest me as a fierce lion: and again thou shewest thyself marvellous upon me."],
    ['Job','Job','10','017','17',"Thou renewest thy witnesses against me, and increasest thine indignation upon me; changes and war \emph{are} against me."],
    ['Job','Job','10','018','18',"Wherefore then hast thou brought me forth out of the womb? Oh that I had given up the ghost, and no eye had seen me!"],
    ['Job','Job','10','019','19',"I should have been as though I had not been; I should have been carried from the womb to the grave."],
    ['Job','Job','10','020','20',"\emph{Are} not my days few? cease \emph{then,} \emph{and} let me alone, that I may take comfort a little,"],
    ['Job','Job','10','021','21',"Before I go \emph{whence} I shall not return, \emph{even} to the land of darkness and the shadow of death;"],
    ['Job','Job','10','022','22',"A land of darkness, as darkness \emph{itself;} \emph{and} of the shadow of death, without any order, and \emph{where} the light \emph{is} as darkness."]
]

Job11 = [
    ['Job','Job','11','001','1',"Then answered Zophar the Naamathite, and said,"],
    ['Job','Job','11','002','2',"Should not the multitude of words be answered? and should a man full of talk be justified?"],
    ['Job','Job','11','003','3',"Should thy lies make men hold their peace? and when thou mockest, shall no man make thee ashamed?"],
    ['Job','Job','11','004','4',"For thou hast said, My doctrine \emph{is} pure, and I am clean in thine eyes."],
    ['Job','Job','11','005','5',"But oh that God would speak, and open his lips against thee;"],
    ['Job','Job','11','006','6',"And that he would shew thee the secrets of wisdom, that \emph{they} \emph{are} double to that which is! Know therefore that God exacteth of thee \emph{less} than thine iniquity \emph{deserveth}."],
    ['Job','Job','11','007','7',"Canst thou by searching find out God? canst thou find out the Almighty unto perfection?"],
    ['Job','Job','11','008','8',"\emph{It} \emph{is} as high as heaven; what canst thou do? deeper than hell; what canst thou know?"],
    ['Job','Job','11','009','9',"The measure thereof \emph{is} longer than the earth, and broader than the sea."],
    ['Job','Job','11','010','10',"If he cut off, and shut up, or gather together, then who can hinder him?"],
    ['Job','Job','11','011','11',"For he knoweth vain men: he seeth wickedness also; will he not then consider \emph{it}?"],
    ['Job','Job','11','012','12',"For vain man would be wise, though man be born \emph{like} a wild ass's colt."],
    ['Job','Job','11','013','13',"If thou prepare thine heart, and stretch out thine hands toward him;"],
    ['Job','Job','11','014','14',"If iniquity \emph{be} in thine hand, put it far away, and let not wickedness dwell in thy tabernacles."],
    ['Job','Job','11','015','15',"For then shalt thou lift up thy face without spot; yea, thou shalt be stedfast, and shalt not fear:"],
    ['Job','Job','11','016','16',"Because thou shalt forget \emph{thy} misery, \emph{and} remember \emph{it} as waters \emph{that} pass away:"],
    ['Job','Job','11','017','17',"And \emph{thine} age shall be clearer than the noonday; thou shalt shine forth, thou shalt be as the morning."],
    ['Job','Job','11','018','18',"And thou shalt be secure, because there is hope; yea, thou shalt dig \emph{about} \emph{thee,} \emph{and} thou shalt take thy rest in safety."],
    ['Job','Job','11','019','19',"Also thou shalt lie down, and none shall make \emph{thee} afraid; yea, many shall make suit unto thee."],
    ['Job','Job','11','020','20',"But the eyes of the wicked shall fail, and they shall not escape, and their hope \emph{shall} \emph{be} \emph{as} the giving up of the ghost."]
]

Job12 = [
    ['Job','Job','12','001','1',"And Job answered and said,"],
    ['Job','Job','12','002','2',"No doubt but ye \emph{are} the people, and wisdom shall die with you."],
    ['Job','Job','12','003','3',"But I have understanding as well as you; I \emph{am} not inferior to you: yea, who knoweth not such things as these?"],
    ['Job','Job','12','004','4',"I am \emph{as} one mocked of his neighbour, who calleth upon God, and he answereth him: the just upright \emph{man} \emph{is} laughed to scorn."],
    ['Job','Job','12','005','5',"He that is ready to slip with \emph{his} feet \emph{is} \emph{as} a lamp despised in the thought of him that is at ease."],
    ['Job','Job','12','006','6',"The tabernacles of robbers prosper, and they that provoke God are secure; into whose hand God bringeth \emph{abundantly}."],
    ['Job','Job','12','007','7',"But ask now the beasts, and they shall teach thee; and the fowls of the air, and they shall tell thee:"],
    ['Job','Job','12','008','8',"Or speak to the earth, and it shall teach thee: and the fishes of the sea shall declare unto thee."],
    ['Job','Job','12','009','9',"Who knoweth not in all these that the hand of the LORD hath wrought this?"],
    ['Job','Job','12','010','10',"In whose hand \emph{is} the soul of every living thing, and the breath of all mankind."],
    ['Job','Job','12','011','11',"Doth not the ear try words? and the mouth taste his meat?"],
    ['Job','Job','12','012','12',"With the ancient \emph{is} wisdom; and in length of days understanding."],
    ['Job','Job','12','013','13',"With him \emph{is} wisdom and strength, he hath counsel and understanding."],
    ['Job','Job','12','014','14',"Behold, he breaketh down, and it cannot be built again: he shutteth up a man, and there can be no opening."],
    ['Job','Job','12','015','15',"Behold, he withholdeth the waters, and they dry up: also he sendeth them out, and they overturn the earth."],
    ['Job','Job','12','016','16',"With him \emph{is} strength and wisdom: the deceived and the deceiver \emph{are} his."],
    ['Job','Job','12','017','17',"He leadeth counsellors away spoiled, and maketh the judges fools."],
    ['Job','Job','12','018','18',"He looseth the bond of kings, and girdeth their loins with a girdle."],
    ['Job','Job','12','019','19',"He leadeth princes away spoiled, and overthroweth the mighty."],
    ['Job','Job','12','020','20',"He removeth away the speech of the trusty, and taketh away the understanding of the aged."],
    ['Job','Job','12','021','21',"He poureth contempt upon princes, and weakeneth the strength of the mighty."],
    ['Job','Job','12','022','22',"He discovereth deep things out of darkness, and bringeth out to light the shadow of death."],
    ['Job','Job','12','023','23',"He increaseth the nations, and destroyeth them: he enlargeth the nations, and straiteneth them \emph{again}."],
    ['Job','Job','12','024','24',"He taketh away the heart of the chief of the people of the earth, and causeth them to wander in a wilderness \emph{where} \emph{there} \emph{is} no way."],
    ['Job','Job','12','025','25',"They grope in the dark without light, and he maketh them to stagger like \emph{a} drunken \emph{man}."]
]

Job13 = [
    ['Job','Job','13','001','1',"Lo, mine eye hath seen all \emph{this}, mine ear hath heard and understood it."],
    ['Job','Job','13','002','2',"What ye know, \emph{the} \emph{same} do I know also: I \emph{am} not inferior unto you."],
    ['Job','Job','13','003','3',"Surely I would speak to the Almighty, and I desire to reason with God."],
    ['Job','Job','13','004','4',"But ye \emph{are} forgers of lies, ye \emph{are} all physicians of no value."],
    ['Job','Job','13','005','5',"O that ye would altogether hold your peace! and it should be your wisdom."],
    ['Job','Job','13','006','6',"Hear now my reasoning, and hearken to the pleadings of my lips."],
    ['Job','Job','13','007','7',"Will ye speak wickedly for God? and talk deceitfully for him?"],
    ['Job','Job','13','008','8',"Will ye accept his person? will ye contend for God?"],
    ['Job','Job','13','009','9',"Is it good that he should search you out? or as one man mocketh another, do ye \emph{so} mock him?"],
    ['Job','Job','13','010','10',"He will surely reprove you, if ye do secretly accept persons."],
    ['Job','Job','13','011','11',"Shall not his excellency make you afraid? and his dread fall upon you?"],
    ['Job','Job','13','012','12',"Your remembrances \emph{are} like unto ashes, your bodies to bodies of clay."],
    ['Job','Job','13','013','13',"Hold your peace, let me alone, that I may speak, and let come on me what \emph{will}."],
    ['Job','Job','13','014','14',"Wherefore do I take my flesh in my teeth, and put my life in mine hand?"],
    ['Job','Job','13','015','15',"Though he slay me, yet will I trust in him: but I will maintain mine own ways before him."],
    ['Job','Job','13','016','16',"He also \emph{shall} \emph{be} my salvation: for an hypocrite shall not come before him."],
    ['Job','Job','13','017','17',"Hear diligently my speech, and my declaration with your ears."],
    ['Job','Job','13','018','18',"Behold now, I have ordered \emph{my} cause; I know that I shall be justified."],
    ['Job','Job','13','019','19',"Who \emph{is} he \emph{that} will plead with me? for now, if I hold my tongue, I shall give up the ghost."],
    ['Job','Job','13','020','20',"Only do not two \emph{things} unto me: then will I not hide myself from thee."],
    ['Job','Job','13','021','21',"Withdraw thine hand far from me: and let not thy dread make me afraid."],
    ['Job','Job','13','022','22',"Then call thou, and I will answer: or let me speak, and answer thou me."],
    ['Job','Job','13','023','23',"How many \emph{are} mine iniquities and sins? make me to know my transgression and my sin."],
    ['Job','Job','13','024','24',"Wherefore hidest thou thy face, and holdest me for thine enemy?"],
    ['Job','Job','13','025','25',"Wilt thou break a leaf driven to and fro? and wilt thou pursue the dry stubble?"],
    ['Job','Job','13','026','26',"For thou writest bitter things against me, and makest me to possess the iniquities of my youth."],
    ['Job','Job','13','027','27',"Thou puttest my feet also in the stocks, and lookest narrowly unto all my paths; thou settest a print upon the heels of my feet."],
    ['Job','Job','13','028','28',"And he, as a rotten thing, consumeth, as a garment that is moth eaten."]
]

Job14 = [
    ['Job','Job','14','001','1',"Man \emph{that} \emph{is} born of a woman \emph{is} of few days, and full of trouble."],
    ['Job','Job','14','002','2',"He cometh forth like a flower, and is cut down: he fleeth also as a shadow, and continueth not."],
    ['Job','Job','14','003','3',"And dost thou open thine eyes upon such an one, and bringest me into judgment with thee?"],
    ['Job','Job','14','004','4',"Who can bring a clean \emph{thing} out of an unclean? not one."],
    ['Job','Job','14','005','5',"Seeing his days \emph{are} determined, the number of his months \emph{are} with thee, thou hast appointed his bounds that he cannot pass;"],
    ['Job','Job','14','006','6',"Turn from him, that he may rest, till he shall accomplish, as an hireling, his day."],
    ['Job','Job','14','007','7',"For there is hope of a tree, if it be cut down, that it will sprout again, and that the tender branch thereof will not cease."],
    ['Job','Job','14','008','8',"Though the root thereof wax old in the earth, and the stock thereof die in the ground;"],
    ['Job','Job','14','009','9',"\emph{Yet} through the scent of water it will bud, and bring forth boughs like a plant."],
    ['Job','Job','14','010','10',"But man dieth, and wasteth away: yea, man giveth up the ghost, and where \emph{is} he?"],
    ['Job','Job','14','011','11',"\emph{As} the waters fail from the sea, and the flood decayeth and drieth up:"],
    ['Job','Job','14','012','12',"So man lieth down, and riseth not: till the heavens \emph{be} no more, they shall not awake, nor be raised out of their sleep."],
    ['Job','Job','14','013','13',"O that thou wouldest hide me in the grave, that thou wouldest keep me secret, until thy wrath be past, that thou wouldest appoint me a set time, and remember me!"],
    ['Job','Job','14','014','14',"If a man die, shall he live \emph{again}? all the days of my appointed time will I wait, till my change come."],
    ['Job','Job','14','015','15',"Thou shalt call, and I will answer thee: thou wilt have a desire to the work of thine hands."],
    ['Job','Job','14','016','16',"For now thou numberest my steps: dost thou not watch over my sin?"],
    ['Job','Job','14','017','17',"My transgression \emph{is} sealed up in a bag, and thou sewest up mine iniquity."],
    ['Job','Job','14','018','18',"And surely the mountain falling cometh to nought, and the rock is removed out of his place."],
    ['Job','Job','14','019','19',"The waters wear the stones: thou washest away the things which grow \emph{out} of the dust of the earth; and thou destroyest the hope of man."],
    ['Job','Job','14','020','20',"Thou prevailest for ever against him, and he passeth: thou changest his countenance, and sendest him away."],
    ['Job','Job','14','021','21',"His sons come to honour, and he knoweth \emph{it} not; and they are brought low, but he perceiveth \emph{it} not of them."],
    ['Job','Job','14','022','22',"But his flesh upon him shall have pain, and his soul within him shall mourn."]
]

Job15 = [
    ['Job','Job','15','001','1',"Then answered Eliphaz the Temanite, and said,"],
    ['Job','Job','15','002','2',"Should a wise man utter vain knowledge, and fill his belly with the east wind?"],
    ['Job','Job','15','003','3',"Should he reason with unprofitable talk? or with speeches wherewith he can do no good?"],
    ['Job','Job','15','004','4',"Yea, thou castest off fear, and restrainest prayer before God."],
    ['Job','Job','15','005','5',"For thy mouth uttereth thine iniquity, and thou choosest the tongue of the crafty."],
    ['Job','Job','15','006','6',"Thine own mouth condemneth thee, and not I: yea, thine own lips testify against thee."],
    ['Job','Job','15','007','7',"\emph{Art} thou the first man \emph{that} was born? or wast thou made before the hills?"],
    ['Job','Job','15','008','8',"Hast thou heard the secret of God? and dost thou restrain wisdom to thyself?"],
    ['Job','Job','15','009','9',"What knowest thou, that we know not? \emph{what} understandest thou, which \emph{is} not in us?"],
    ['Job','Job','15','010','10',"With us \emph{are} both the grayheaded and very aged men, much elder than thy father."],
    ['Job','Job','15','011','11',"\emph{Are} the consolations of God small with thee? is there any secret thing with thee?"],
    ['Job','Job','15','012','12',"Why doth thine heart carry thee away? and what do thy eyes wink at,"],
    ['Job','Job','15','013','13',"That thou turnest thy spirit against God, and lettest \emph{such} words go out of thy mouth?"],
    ['Job','Job','15','014','14',"What \emph{is} man, that he should be clean? and \emph{he} \emph{which} \emph{is} born of a woman, that he should be righteous?"],
    ['Job','Job','15','015','15',"Behold, he putteth no trust in his saints; yea, the heavens are not clean in his sight."],
    ['Job','Job','15','016','16',"How much more abominable and filthy \emph{is} man, which drinketh iniquity like water?"],
    ['Job','Job','15','017','17',"I will shew thee, hear me; and that \emph{which} I have seen I will declare;"],
    ['Job','Job','15','018','18',"Which wise men have told from their fathers, and have not hid \emph{it}:"],
    ['Job','Job','15','019','19',"Unto whom alone the earth was given, and no stranger passed among them."],
    ['Job','Job','15','020','20',"The wicked man travaileth with pain all \emph{his} days, and the number of years is hidden to the oppressor."],
    ['Job','Job','15','021','21',"A dreadful sound \emph{is} in his ears: in prosperity the destroyer shall come upon him."],
    ['Job','Job','15','022','22',"He believeth not that he shall return out of darkness, and he is waited for of the sword."],
    ['Job','Job','15','023','23',"He wandereth abroad for bread, \emph{saying}, Where \emph{is} \emph{it}? he knoweth that the day of darkness is ready at his hand."],
    ['Job','Job','15','024','24',"Trouble and anguish shall make him afraid; they shall prevail against him, as a king ready to the battle."],
    ['Job','Job','15','025','25',"For he stretcheth out his hand against God, and strengtheneth himself against the Almighty."],
    ['Job','Job','15','026','26',"He runneth upon him, \emph{even} on \emph{his} neck, upon the thick bosses of his bucklers:"],
    ['Job','Job','15','027','27',"Because he covereth his face with his fatness, and maketh collops of fat on \emph{his} flanks."],
    ['Job','Job','15','028','28',"And he dwelleth in desolate cities, \emph{and} in houses which no man inhabiteth, which are ready to become heaps."],
    ['Job','Job','15','029','29',"He shall not be rich, neither shall his substance continue, neither shall he prolong the perfection thereof upon the earth."],
    ['Job','Job','15','030','30',"He shall not depart out of darkness; the flame shall dry up his branches, and by the breath of his mouth shall he go away."],
    ['Job','Job','15','031','31',"Let not him that is deceived trust in vanity: for vanity shall be his recompence."],
    ['Job','Job','15','032','32',"It shall be accomplished before his time, and his branch shall not be green."],
    ['Job','Job','15','033','33',"He shall shake off his unripe grape as the vine, and shall cast off his flower as the olive."],
    ['Job','Job','15','034','34',"For the congregation of hypocrites \emph{shall} \emph{be} desolate, and fire shall consume the tabernacles of bribery."],
    ['Job','Job','15','035','35',"They conceive mischief, and bring forth vanity, and their belly prepareth deceit."]
]

Job16 = [
    ['Job','Job','16','001','1',"Then Job answered and said,"],
    ['Job','Job','16','002','2',"I have heard many such things: miserable comforters \emph{are} ye all."],
    ['Job','Job','16','003','3',"Shall vain words have an end? or what emboldeneth thee that thou answerest?"],
    ['Job','Job','16','004','4',"I also could speak as ye \emph{do}: if your soul were in my soul's stead, I could heap up words against you, and shake mine head at you."],
    ['Job','Job','16','005','5',"\emph{But} I would strengthen you with my mouth, and the moving of my lips should asswage \emph{your} \emph{grief}."],
    ['Job','Job','16','006','6',"Though I speak, my grief is not asswaged: and \emph{though} I forbear, what am I eased?"],
    ['Job','Job','16','007','7',"But now he hath made me weary: thou hast made desolate all my company."],
    ['Job','Job','16','008','8',"And thou hast filled me with wrinkles, \emph{which} is a witness \emph{against} \emph{me}: and my leanness rising up in me beareth witness to my face."],
    ['Job','Job','16','009','9',"He teareth \emph{me} in his wrath, who hateth me: he gnasheth upon me with his teeth; mine enemy sharpeneth his eyes upon me."],
    ['Job','Job','16','010','10',"They have gaped upon me with their mouth; they have smitten me upon the cheek reproachfully; they have gathered themselves together against me."],
    ['Job','Job','16','011','11',"God hath delivered me to the ungodly, and turned me over into the hands of the wicked."],
    ['Job','Job','16','012','12',"I was at ease, but he hath broken me asunder: he hath also taken \emph{me} by my neck, and shaken me to pieces, and set me up for his mark."],
    ['Job','Job','16','013','13',"His archers compass me round about, he cleaveth my reins asunder, and doth not spare; he poureth out my gall upon the ground."],
    ['Job','Job','16','014','14',"He breaketh me with breach upon breach, he runneth upon me like a giant."],
    ['Job','Job','16','015','15',"I have sewed sackcloth upon my skin, and defiled my horn in the dust."],
    ['Job','Job','16','016','16',"My face is foul with weeping, and on my eyelids \emph{is} the shadow of death;"],
    ['Job','Job','16','017','17',"Not for \emph{any} injustice in mine hands: also my prayer \emph{is} pure."],
    ['Job','Job','16','018','18',"O earth, cover not thou my blood, and let my cry have no place."],
    ['Job','Job','16','019','19',"Also now, behold, my witness \emph{is} in heaven, and my record \emph{is} on high."],
    ['Job','Job','16','020','20',"My friends scorn me: \emph{but} mine eye poureth out \emph{tears} unto God."],
    ['Job','Job','16','021','21',"O that one might plead for a man with God, as a man \emph{pleadeth} for his neighbour!"],
    ['Job','Job','16','022','22',"When a few years are come, then I shall go the way \emph{whence} I shall not return."]
]

Job17 = [
    ['Job','Job','17','001','1',"My breath is corrupt, my days are extinct, the graves \emph{are} \emph{ready} for me."],
    ['Job','Job','17','002','2',"\emph{Are} \emph{there} not mockers with me? and doth not mine eye continue in their provocation?"],
    ['Job','Job','17','003','3',"Lay down now, put me in a surety with thee; who \emph{is} he \emph{that} will strike hands with me?"],
    ['Job','Job','17','004','4',"For thou hast hid their heart from understanding: therefore shalt thou not exalt \emph{them}."],
    ['Job','Job','17','005','5',"He that speaketh flattery to \emph{his} friends, even the eyes of his children shall fail."],
    ['Job','Job','17','006','6',"He hath made me also a byword of the people; and aforetime I was as a tabret."],
    ['Job','Job','17','007','7',"Mine eye also is dim by reason of sorrow, and all my members \emph{are} as a shadow."],
    ['Job','Job','17','008','8',"Upright \emph{men} shall be astonied at this, and the innocent shall stir up himself against the hypocrite."],
    ['Job','Job','17','009','9',"The righteous also shall hold on his way, and he that hath clean hands shall be stronger and stronger."],
    ['Job','Job','17','010','10',"But as for you all, do ye return, and come now: for I cannot find \emph{one} wise \emph{man} among you."],
    ['Job','Job','17','011','11',"My days are past, my purposes are broken off, \emph{even} the thoughts of my heart."],
    ['Job','Job','17','012','12',"They change the night into day: the light \emph{is} short because of darkness."],
    ['Job','Job','17','013','13',"If I wait, the grave \emph{is} mine house: I have made my bed in the darkness."],
    ['Job','Job','17','014','14',"I have said to corruption, Thou \emph{art} my father: to the worm, \emph{Thou} \emph{art} my mother, and my sister."],
    ['Job','Job','17','015','15',"And where \emph{is} now my hope? as for my hope, who shall see it?"],
    ['Job','Job','17','016','16',"They shall go down to the bars of the pit, when \emph{our} rest together \emph{is} in the dust."]
]

Job18 = [
    ['Job','Job','18','001','1',"Then answered Bildad the Shuhite, and said,"],
    ['Job','Job','18','002','2',"How long \emph{will} \emph{it} \emph{be} \emph{ere} ye make an end of words? mark, and afterwards we will speak."],
    ['Job','Job','18','003','3',"Wherefore are we counted as beasts, \emph{and} reputed vile in your sight?"],
    ['Job','Job','18','004','4',"He teareth himself in his anger: shall the earth be forsaken for thee? and shall the rock be removed out of his place?"],
    ['Job','Job','18','005','5',"Yea, the light of the wicked shall be put out, and the spark of his fire shall not shine."],
    ['Job','Job','18','006','6',"The light shall be dark in his tabernacle, and his candle shall be put out with him."],
    ['Job','Job','18','007','7',"The steps of his strength shall be straitened, and his own counsel shall cast him down."],
    ['Job','Job','18','008','8',"For he is cast into a net by his own feet, and he walketh upon a snare."],
    ['Job','Job','18','009','9',"The gin shall take \emph{him} by the heel, \emph{and} the robber shall prevail against him."],
    ['Job','Job','18','010','10',"The snare \emph{is} laid for him in the ground, and a trap for him in the way."],
    ['Job','Job','18','011','11',"Terrors shall make him afraid on every side, and shall drive him to his feet."],
    ['Job','Job','18','012','12',"His strength shall be hungerbitten, and destruction \emph{shall} \emph{be} ready at his side."],
    ['Job','Job','18','013','13',"It shall devour the strength of his skin: \emph{even} the firstborn of death shall devour his strength."],
    ['Job','Job','18','014','14',"His confidence shall be rooted out of his tabernacle, and it shall bring him to the king of terrors."],
    ['Job','Job','18','015','15',"It shall dwell in his tabernacle, because \emph{it} \emph{is} none of his: brimstone shall be scattered upon his habitation."],
    ['Job','Job','18','016','16',"His roots shall be dried up beneath, and above shall his branch be cut off."],
    ['Job','Job','18','017','17',"His remembrance shall perish from the earth, and he shall have no name in the street."],
    ['Job','Job','18','018','18',"He shall be driven from light into darkness, and chased out of the world."],
    ['Job','Job','18','019','19',"He shall neither have son nor nephew among his people, nor any remaining in his dwellings."],
    ['Job','Job','18','020','20',"They that come after \emph{him} shall be astonied at his day, as they that went before were affrighted."],
    ['Job','Job','18','021','21',"Surely such \emph{are} the dwellings of the wicked, and this \emph{is} the place \emph{of} \emph{him} \emph{that} knoweth not God."]
]

Job19 = [
    ['Job','Job','19','001','1',"Then Job answered and said,"],
    ['Job','Job','19','002','2',"How long will ye vex my soul, and break me in pieces with words?"],
    ['Job','Job','19','003','3',"These ten times have ye reproached me: ye are not ashamed \emph{that} ye make yourselves strange to me."],
    ['Job','Job','19','004','4',"And be it indeed \emph{that} I have erred, mine error remaineth with myself."],
    ['Job','Job','19','005','5',"If indeed ye will magnify \emph{yourselves} against me, and plead against me my reproach:"],
    ['Job','Job','19','006','6',"Know now that God hath overthrown me, and hath compassed me with his net."],
    ['Job','Job','19','007','7',"Behold, I cry out of wrong, but I am not heard: I cry aloud, but \emph{there} \emph{is} no judgment."],
    ['Job','Job','19','008','8',"He hath fenced up my way that I cannot pass, and he hath set darkness in my paths."],
    ['Job','Job','19','009','9',"He hath stripped me of my glory, and taken the crown \emph{from} my head."],
    ['Job','Job','19','010','10',"He hath destroyed me on every side, and I am gone: and mine hope hath he removed like a tree."],
    ['Job','Job','19','011','11',"He hath also kindled his wrath against me, and he counteth me unto him as \emph{one} \emph{of} his enemies."],
    ['Job','Job','19','012','12',"His troops come together, and raise up their way against me, and encamp round about my tabernacle."],
    ['Job','Job','19','013','13',"He hath put my brethren far from me, and mine acquaintance are verily estranged from me."],
    ['Job','Job','19','014','14',"My kinsfolk have failed, and my familiar friends have forgotten me."],
    ['Job','Job','19','015','15',"They that dwell in mine house, and my maids, count me for a stranger: I am an alien in their sight."],
    ['Job','Job','19','016','16',"I called my servant, and he gave \emph{me} no answer; I intreated him with my mouth."],
    ['Job','Job','19','017','17',"My breath is strange to my wife, though I intreated for the children's \emph{sake} of mine own body."],
    ['Job','Job','19','018','18',"Yea, young children despised me; I arose, and they spake against me."],
    ['Job','Job','19','019','19',"All my inward friends abhorred me: and they whom I loved are turned against me."],
    ['Job','Job','19','020','20',"My bone cleaveth to my skin and to my flesh, and I am escaped with the skin of my teeth."],
    ['Job','Job','19','021','21',"Have pity upon me, have pity upon me, O ye my friends; for the hand of God hath touched me."],
    ['Job','Job','19','022','22',"Why do ye persecute me as God, and are not satisfied with my flesh?"],
    ['Job','Job','19','023','23',"Oh that my words were now written! oh that they were printed in a book!"],
    ['Job','Job','19','024','24',"That they were graven with an iron pen and lead in the rock for ever!"],
    ['Job','Job','19','025','25',"For I know \emph{that} my redeemer liveth, and \emph{that} he shall stand at the latter \emph{day} upon the earth:"],
    ['Job','Job','19','026','26',"And \emph{though} after my skin \emph{worms} destroy this \emph{body}, yet in my flesh shall I see God:"],
    ['Job','Job','19','027','27',"Whom I shall see for myself, and mine eyes shall behold, and not another; \emph{though} my reins be consumed within me."],
    ['Job','Job','19','028','28',"But ye should say, Why persecute we him, seeing the root of the matter is found in me?"],
    ['Job','Job','19','029','29',"Be ye afraid of the sword: for wrath \emph{bringeth} the punishments of the sword, that ye may know \emph{there} \emph{is} a judgment."]
]

Job20 = [
    ['Job','Job','20','001','1',"Then answered Zophar the Naamathite, and said,"],
    ['Job','Job','20','002','2',"Therefore do my thoughts cause me to answer, and for \emph{this} I make haste."],
    ['Job','Job','20','003','3',"I have heard the check of my reproach, and the spirit of my understanding causeth me to answer."],
    ['Job','Job','20','004','4',"Knowest thou \emph{not} this of old, since man was placed upon earth,"],
    ['Job','Job','20','005','5',"That the triumphing of the wicked \emph{is} short, and the joy of the hypocrite \emph{but} for a moment?"],
    ['Job','Job','20','006','6',"Though his excellency mount up to the heavens, and his head reach unto the clouds;"],
    ['Job','Job','20','007','7',"\emph{Yet} he shall perish for ever like his own dung: they which have seen him shall say, Where \emph{is} he?"],
    ['Job','Job','20','008','8',"He shall fly away as a dream, and shall not be found: yea, he shall be chased away as a vision of the night."],
    ['Job','Job','20','009','9',"The eye also \emph{which} saw him shall \emph{see} \emph{him} no more; neither shall his place any more behold him."],
    ['Job','Job','20','010','10',"His children shall seek to please the poor, and his hands shall restore their goods."],
    ['Job','Job','20','011','11',"His bones are full \emph{of} \emph{the} \emph{sin} of his youth, which shall lie down with him in the dust."],
    ['Job','Job','20','012','12',"Though wickedness be sweet in his mouth, \emph{though} he hide it under his tongue;"],
    ['Job','Job','20','013','13',"\emph{Though} he spare it, and forsake it not; but keep it still within his mouth:"],
    ['Job','Job','20','014','14',"\emph{Yet} his meat in his bowels is turned, \emph{it} \emph{is} the gall of asps within him."],
    ['Job','Job','20','015','15',"He hath swallowed down riches, and he shall vomit them up again: God shall cast them out of his belly."],
    ['Job','Job','20','016','16',"He shall suck the poison of asps: the viper's tongue shall slay him."],
    ['Job','Job','20','017','17',"He shall not see the rivers, the floods, the brooks of honey and butter."],
    ['Job','Job','20','018','18',"That which he laboured for shall he restore, and shall not swallow \emph{it} down: according to \emph{his} substance \emph{shall} the restitution \emph{be}, and he shall not rejoice \emph{therein}."],
    ['Job','Job','20','019','19',"Because he hath oppressed \emph{and} hath forsaken the poor; \emph{because} he hath violently taken away an house which he builded not;"],
    ['Job','Job','20','020','20',"Surely he shall not feel quietness in his belly, he shall not save of that which he desired."],
    ['Job','Job','20','021','21',"There shall none of his meat be left; therefore shall no man look for his goods."],
    ['Job','Job','20','022','22',"In the fulness of his sufficiency he shall be in straits: every hand of the wicked shall come upon him."],
    ['Job','Job','20','023','23',"\emph{When} he is about to fill his belly, \emph{God} shall cast the fury of his wrath upon him, and shall rain \emph{it} upon him while he is eating."],
    ['Job','Job','20','024','24',"He shall flee from the iron weapon, \emph{and} the bow of steel shall strike him through."],
    ['Job','Job','20','025','25',"It is drawn, and cometh out of the body; yea, the glittering sword cometh out of his gall: terrors \emph{are} upon him."],
    ['Job','Job','20','026','26',"All darkness \emph{shall} \emph{be} hid in his secret places: a fire not blown shall consume him; it shall go ill with him that is left in his tabernacle."],
    ['Job','Job','20','027','27',"The heaven shall reveal his iniquity; and the earth shall rise up against him."],
    ['Job','Job','20','028','28',"The increase of his house shall depart, \emph{and} \emph{his} \emph{goods} shall flow away in the day of his wrath."],
    ['Job','Job','20','029','29',"This \emph{is} the portion of a wicked man from God, and the heritage appointed unto him by God."]
]

Job21 = [
    ['Job','Job','21','001','1',"But Job answered and said,"],
    ['Job','Job','21','002','2',"Hear diligently my speech, and let this be your consolations."],
    ['Job','Job','21','003','3',"Suffer me that I may speak; and after that I have spoken, mock on."],
    ['Job','Job','21','004','4',"As for me, \emph{is} my complaint to man? and if \emph{it} \emph{were} \emph{so}, why should not my spirit be troubled?"],
    ['Job','Job','21','005','5',"Mark me, and be astonished, and lay \emph{your} hand upon \emph{your} mouth."],
    ['Job','Job','21','006','6',"Even when I remember I am afraid, and trembling taketh hold on my flesh."],
    ['Job','Job','21','007','7',"Wherefore do the wicked live, become old, yea, are mighty in power?"],
    ['Job','Job','21','008','8',"Their seed is established in their sight with them, and their offspring before their eyes."],
    ['Job','Job','21','009','9',"Their houses \emph{are} safe from fear, neither \emph{is} the rod of God upon them."],
    ['Job','Job','21','010','10',"Their bull gendereth, and faileth not; their cow calveth, and casteth not her calf."],
    ['Job','Job','21','011','11',"They send forth their little ones like a flock, and their children dance."],
    ['Job','Job','21','012','12',"They take the timbrel and harp, and rejoice at the sound of the organ."],
    ['Job','Job','21','013','13',"They spend their days in wealth, and in a moment go down to the grave."],
    ['Job','Job','21','014','14',"Therefore they say unto God, Depart from us; for we desire not the knowledge of thy ways."],
    ['Job','Job','21','015','15',"What \emph{is} the Almighty, that we should serve him? and what profit should we have, if we pray unto him?"],
    ['Job','Job','21','016','16',"Lo, their good \emph{is} not in their hand: the counsel of the wicked is far from me."],
    ['Job','Job','21','017','17',"How oft is the candle of the wicked put out! and \emph{how} \emph{oft} cometh their destruction upon them! \emph{God} distributeth sorrows in his anger."],
    ['Job','Job','21','018','18',"They are as stubble before the wind, and as chaff that the storm carrieth away."],
    ['Job','Job','21','019','19',"God layeth up his iniquity for his children: he rewardeth him, and he shall know \emph{it}."],
    ['Job','Job','21','020','20',"His eyes shall see his destruction, and he shall drink of the wrath of the Almighty."],
    ['Job','Job','21','021','21',"For what pleasure \emph{hath} he in his house after him, when the number of his months is cut off in the midst?"],
    ['Job','Job','21','022','22',"Shall \emph{any} teach God knowledge? seeing he judgeth those that are high."],
    ['Job','Job','21','023','23',"One dieth in his full strength, being wholly at ease and quiet."],
    ['Job','Job','21','024','24',"His breasts are full of milk, and his bones are moistened with marrow."],
    ['Job','Job','21','025','25',"And another dieth in the bitterness of his soul, and never eateth with pleasure."],
    ['Job','Job','21','026','26',"They shall lie down alike in the dust, and the worms shall cover them."],
    ['Job','Job','21','027','27',"Behold, I know your thoughts, and the devices \emph{which} ye wrongfully imagine against me."],
    ['Job','Job','21','028','28',"For ye say, Where \emph{is} the house of the prince? and where \emph{are} the dwelling places of the wicked?"],
    ['Job','Job','21','029','29',"Have ye not asked them that go by the way? and do ye not know their tokens,"],
    ['Job','Job','21','030','30',"That the wicked is reserved to the day of destruction? they shall be brought forth to the day of wrath."],
    ['Job','Job','21','031','31',"Who shall declare his way to his face? and who shall repay him \emph{what} he hath done?"],
    ['Job','Job','21','032','32',"Yet shall he be brought to the grave, and shall remain in the tomb."],
    ['Job','Job','21','033','33',"The clods of the valley shall be sweet unto him, and every man shall draw after him, as \emph{there} \emph{are} innumerable before him."],
    ['Job','Job','21','034','34',"How then comfort ye me in vain, seeing in your answers there remaineth falsehood?"]
]

Job22 = [
    ['Job','Job','22','001','1',"Then Eliphaz the Temanite answered and said,"],
['Job','Job','22','002','2',"Can a man be profitable unto God, as he that is wise may be profitable unto himself?"],
['Job','Job','22','003','3',"\emph{Is} \emph{it} any pleasure to the Almighty, that thou art righteous? or \emph{is} \emph{it} gain \emph{to} \emph{him}, that thou makest thy ways perfect?"],
['Job','Job','22','004','4',"Will he reprove thee for fear of thee? will he enter with thee into judgment?"],
['Job','Job','22','005','5',"\emph{Is} not thy wickedness great? and thine iniquities infinite?"],
['Job','Job','22','006','6',"For thou hast taken a pledge from thy brother for nought, and stripped the naked of their clothing."],
['Job','Job','22','007','7',"Thou hast not given water to the weary to drink, and thou hast withholden bread from the hungry."],
['Job','Job','22','008','8',"But \emph{as} \emph{for} the mighty man, he had the earth; and the honourable man dwelt in it."],
['Job','Job','22','009','9',"Thou hast sent widows away empty, and the arms of the fatherless have been broken."],
['Job','Job','22','010','10',"Therefore snares \emph{are} round about thee, and sudden fear troubleth thee;"],
['Job','Job','22','011','11',"Or darkness, \emph{that} thou canst not see; and abundance of waters cover thee."],
['Job','Job','22','012','12',"\emph{Is} not God in the height of heaven? and behold the height of the stars, how high they are!"],
['Job','Job','22','013','13',"And thou sayest, How doth God know? can he judge through the dark cloud?"],
['Job','Job','22','014','14',"Thick clouds \emph{are} a covering to him, that he seeth not; and he walketh in the circuit of heaven."],
['Job','Job','22','015','15',"Hast thou marked the old way which wicked men have trodden?"],
['Job','Job','22','016','16',"Which were cut down out of time, whose foundation was overflown with a flood:"],
['Job','Job','22','017','17',"Which said unto God, Depart from us: and what can the Almighty do for them?"],
['Job','Job','22','018','18',"Yet he filled their houses with good \emph{things}: but the counsel of the wicked is far from me."],
['Job','Job','22','019','19',"The righteous see \emph{it}, and are glad: and the innocent laugh them to scorn."],
['Job','Job','22','020','20',"Whereas our substance is not cut down, but the remnant of them the fire consumeth."],
['Job','Job','22','021','21',"Acquaint now thyself with him, and be at peace: thereby good shall come unto thee."],
['Job','Job','22','022','22',"Receive, I pray thee, the law from his mouth, and lay up his words in thine heart."],
['Job','Job','22','023','23',"If thou return to the Almighty, thou shalt be built up, thou shalt put away iniquity far from thy tabernacles."],
['Job','Job','22','024','24',"Then shalt thou lay up gold as dust, and the \emph{gold} of Ophir as the stones of the brooks."],
['Job','Job','22','025','25',"Yea, the Almighty shall be thy defence, and thou shalt have plenty of silver."],
['Job','Job','22','026','26',"For then shalt thou have thy delight in the Almighty, and shalt lift up thy face unto God."],
['Job','Job','22','027','27',"Thou shalt make thy prayer unto him, and he shall hear thee, and thou shalt pay thy vows."],
['Job','Job','22','028','28',"Thou shalt also decree a thing, and it shall be established unto thee: and the light shall shine upon thy ways."],
['Job','Job','22','029','29',"When \emph{men} are cast down, then thou shalt say, \emph{There} \emph{is} lifting up; and he shall save the humble person."],
['Job','Job','22','030','30',"He shall deliver the island of the innocent: and it is delivered by the pureness of thine hands."]
]

Job23 = [
    ['Job','Job','23','001','1',"Then Job answered and said,"],
['Job','Job','23','002','2',"Even to day \emph{is} my complaint bitter: my stroke is heavier than my groaning."],
['Job','Job','23','003','3',"Oh that I knew where I might find him! \emph{that} I might come \emph{even} to his seat!"],
['Job','Job','23','004','4',"I would order \emph{my} cause before him, and fill my mouth with arguments."],
['Job','Job','23','005','5',"I would know the words \emph{which} he would answer me, and understand what he would say unto me."],
['Job','Job','23','006','6',"Will he plead against me with \emph{his} great power? No; but he would put \emph{strength} in me."],
['Job','Job','23','007','7',"There the righteous might dispute with him; so should I be delivered for ever from my judge."],
['Job','Job','23','008','8',"Behold, I go forward, but he \emph{is} not \emph{there}; and backward, but I cannot perceive him:"],
['Job','Job','23','009','9',"On the left hand, where he doth work, but I cannot behold \emph{him}: he hideth himself on the right hand, that I cannot see \emph{him}:"],
['Job','Job','23','010','10',"But he knoweth the way that I take: \emph{when} he hath tried me, I shall come forth as gold."],
['Job','Job','23','011','11',"My foot hath held his steps, his way have I kept, and not declined."],
['Job','Job','23','012','12',"Neither have I gone back from the commandment of his lips; I have esteemed the words of his mouth more than my necessary \emph{food}."],
['Job','Job','23','013','13',"But he \emph{is} in one \emph{mind}, and who can turn him? and \emph{what} his soul desireth, even \emph{that} he doeth."],
['Job','Job','23','014','14',"For he performeth \emph{the} \emph{thing} \emph{that} \emph{is} appointed for me: and many such \emph{things} \emph{are} with him."],
['Job','Job','23','015','15',"Therefore am I troubled at his presence: when I consider, I am afraid of him."],
['Job','Job','23','016','16',"For God maketh my heart soft, and the Almighty troubleth me:"],
['Job','Job','23','017','17',"Because I was not cut off before the darkness, \emph{neither} hath he covered the darkness from my face."]
]

Job24 = [
    ['Job','Job','24','001','1',"Why, seeing times are not hidden from the Almighty, do they that know him not see his days?"],
['Job','Job','24','002','2',"\emph{Some} remove the landmarks; they violently take away flocks, and feed \emph{thereof}."],
['Job','Job','24','003','3',"They drive away the ass of the fatherless, they take the widow's ox for a pledge."],
['Job','Job','24','004','4',"They turn the needy out of the way: the poor of the earth hide themselves together."],
['Job','Job','24','005','5',"Behold, \emph{as} wild asses in the desert, go they forth to their work; rising betimes for a prey: the wilderness \emph{yieldeth} food for them \emph{and} for \emph{their} children."],
['Job','Job','24','006','6',"They reap \emph{every} \emph{one} his corn in the field: and they gather the vintage of the wicked."],
['Job','Job','24','007','7',"They cause the naked to lodge without clothing, that \emph{they} \emph{have} no covering in the cold."],
['Job','Job','24','008','8',"They are wet with the showers of the mountains, and embrace the rock for want of a shelter."],
['Job','Job','24','009','9',"They pluck the fatherless from the breast, and take a pledge of the poor."],
['Job','Job','24','010','10',"They cause \emph{him} to go naked without clothing, and they take away the sheaf \emph{from} the hungry;"],
['Job','Job','24','011','11',"\emph{Which} make oil within their walls, \emph{and} tread \emph{their} winepresses, and suffer thirst."],
['Job','Job','24','012','12',"Men groan from out of the city, and the soul of the wounded crieth out: yet God layeth not folly \emph{to} \emph{them}."],
['Job','Job','24','013','13',"They are of those that rebel against the light; they know not the ways thereof, nor abide in the paths thereof."],
['Job','Job','24','014','14',"The murderer rising with the light killeth the poor and needy, and in the night is as a thief."],
['Job','Job','24','015','15',"The eye also of the adulterer waiteth for the twilight, saying, No eye shall see me: and disguiseth \emph{his} face."],
['Job','Job','24','016','16',"In the dark they dig through houses, \emph{which} they had marked for themselves in the daytime: they know not the light."],
['Job','Job','24','017','17',"For the morning \emph{is} to them even as the shadow of death: if \emph{one} know \emph{them,} \emph{they} \emph{are} \emph{in} the terrors of the shadow of death."],
['Job','Job','24','018','18',"He \emph{is} swift as the waters; their portion is cursed in the earth: he beholdeth not the way of the vineyards."],
['Job','Job','24','019','19',"Drought and heat consume the snow waters: \emph{so} \emph{doth} the grave \emph{those} \emph{which} have sinned."],
['Job','Job','24','020','20',"The womb shall forget him; the worm shall feed sweetly on him; he shall be no more remembered; and wickedness shall be broken as a tree."],
['Job','Job','24','021','21',"He evil entreateth the barren \emph{that} beareth not: and doeth not good to the widow."],
['Job','Job','24','022','22',"He draweth also the mighty with his power: he riseth up, and no \emph{man} is sure of life."],
['Job','Job','24','023','23',"\emph{Though} it be given him \emph{to} \emph{be} in safety, whereon he resteth; yet his eyes \emph{are} upon their ways."],
['Job','Job','24','024','24',"They are exalted for a little while, but are gone and brought low; they are taken out of the way as all \emph{other}, and cut off as the tops of the ears of corn."],
['Job','Job','24','025','25',"And if \emph{it} \emph{be} not \emph{so} now, who will make me a liar, and make my speech nothing worth?"]
]

Job25 = [
    ['Job','Job','25','001','1',"Then answered Bildad the Shuhite, and said,"],
['Job','Job','25','002','2',"Dominion and fear \emph{are} with him, he maketh peace in his high places."],
['Job','Job','25','003','3',"Is there any number of his armies? and upon whom doth not his light arise?"],
['Job','Job','25','004','4',"How then can man be justified with God? or how can he be clean \emph{that} \emph{is} born of a woman?"],
['Job','Job','25','005','5',"Behold even to the moon, and it shineth not; yea, the stars are not pure in his sight."],
['Job','Job','25','006','6',"How much less man, \emph{that} \emph{is} a worm? and the son of man, \emph{which} \emph{is} a worm?"]
]

Job26 = [
    ['Job','Job','26','001','1',"But Job answered and said,"],
['Job','Job','26','002','2',"How hast thou helped \emph{him} \emph{that} \emph{is} without power? \emph{how} savest thou the arm \emph{that} \emph{hath} no strength?"],
['Job','Job','26','003','3',"How hast thou counselled \emph{him} \emph{that} \emph{hath} no wisdom? and \emph{how} hast thou plentifully declared the thing as it is?"],
['Job','Job','26','004','4',"To whom hast thou uttered words? and whose spirit came from thee?"],
['Job','Job','26','005','5',"Dead \emph{things} are formed from under the waters, and the inhabitants thereof."],
['Job','Job','26','006','6',"Hell \emph{is} naked before him, and destruction hath no covering."],
['Job','Job','26','007','7',"He stretcheth out the north over the empty place, \emph{and} hangeth the earth upon nothing."],
['Job','Job','26','008','8',"He bindeth up the waters in his thick clouds; and the cloud is not rent under them."],
['Job','Job','26','009','9',"He holdeth back the face of his throne, \emph{and} spreadeth his cloud upon it."],
['Job','Job','26','010','10',"He hath compassed the waters with bounds, until the day and night come to an end."],
['Job','Job','26','011','11',"The pillars of heaven tremble and are astonished at his reproof."],
['Job','Job','26','012','12',"He divideth the sea with his power, and by his understanding he smiteth through the proud."],
['Job','Job','26','013','13',"By his spirit he hath garnished the heavens; his hand hath formed the crooked serpent."],
['Job','Job','26','014','14',"Lo, these \emph{are} parts of his ways: but how little a portion is heard of him? but the thunder of his power who can understand?"]
]

Job27 = [
    ['Job','Job','27','001','1',"Moreover Job continued his parable, and said,"],
['Job','Job','27','002','2',"\emph{As} God liveth, \emph{who} hath taken away my judgment; and the Almighty, \emph{who} hath vexed my soul;"],
['Job','Job','27','003','3',"All the while my breath \emph{is} in me, and the spirit of God \emph{is} in my nostrils;"],
['Job','Job','27','004','4',"My lips shall not speak wickedness, nor my tongue utter deceit."],
['Job','Job','27','005','5',"God forbid that I should justify you: till I die I will not remove mine integrity from me."],
['Job','Job','27','006','6',"My righteousness I hold fast, and will not let it go: my heart shall not reproach \emph{me} so long as I live."],
['Job','Job','27','007','7',"Let mine enemy be as the wicked, and he that riseth up against me as the unrighteous."],
['Job','Job','27','008','8',"For what \emph{is} the hope of the hypocrite, though he hath gained, when God taketh away his soul?"],
['Job','Job','27','009','9',"Will God hear his cry when trouble cometh upon him?"],
['Job','Job','27','010','10',"Will he delight himself in the Almighty? will he always call upon God?"],
['Job','Job','27','011','11',"I will teach you by the hand of God: \emph{that} which \emph{is} with the Almighty will I not conceal."],
['Job','Job','27','012','12',"Behold, all ye yourselves have seen \emph{it}; why then are ye thus altogether vain?"],
['Job','Job','27','013','13',"This \emph{is} the portion of a wicked man with God, and the heritage of oppressors, \emph{which} they shall receive of the Almighty."],
['Job','Job','27','014','14',"If his children be multiplied, \emph{it} \emph{is} for the sword: and his offspring shall not be satisfied with bread."],
['Job','Job','27','015','15',"Those that remain of him shall be buried in death: and his widows shall not weep."],
['Job','Job','27','016','16',"Though he heap up silver as the dust, and prepare raiment as the clay;"],
['Job','Job','27','017','17',"He may prepare \emph{it}, but the just shall put \emph{it} on, and the innocent shall divide the silver."],
['Job','Job','27','018','18',"He buildeth his house as a moth, and as a booth \emph{that} the keeper maketh."],
['Job','Job','27','019','19',"The rich man shall lie down, but he shall not be gathered: he openeth his eyes, and he \emph{is} not."],
['Job','Job','27','020','20',"Terrors take hold on him as waters, a tempest stealeth him away in the night."],
['Job','Job','27','021','21',"The east wind carrieth him away, and he departeth: and as a storm hurleth him out of his place."],
['Job','Job','27','022','22',"For \emph{God} shall cast upon him, and not spare: he would fain flee out of his hand."],
['Job','Job','27','023','23',"\emph{Men} shall clap their hands at him, and shall hiss him out of his place."]
]

Job28 = [
    ['Job','Job','28','001','1',"Surely there is a vein for the silver, and a place for gold \emph{where} they fine \emph{it}."],
['Job','Job','28','002','2',"Iron is taken out of the earth, and brass \emph{is} molten \emph{out} \emph{of} the stone."],
['Job','Job','28','003','3',"He setteth an end to darkness, and searcheth out all perfection: the stones of darkness, and the shadow of death."],
['Job','Job','28','004','4',"The flood breaketh out from the inhabitant; \emph{even} \emph{the} \emph{waters} forgotten of the foot: they are dried up, they are gone away from men."],
['Job','Job','28','005','5',"\emph{As} \emph{for} the earth, out of it cometh bread: and under it is turned up as it were fire."],
['Job','Job','28','006','6',"The stones of it \emph{are} the place of sapphires: and it hath dust of gold."],
['Job','Job','28','007','7',"\emph{There} \emph{is} a path which no fowl knoweth, and which the vulture's eye hath not seen:"],
['Job','Job','28','008','8',"The lion's whelps have not trodden it, nor the fierce lion passed by it."],
['Job','Job','28','009','9',"He putteth forth his hand upon the rock; he overturneth the mountains by the roots."],
['Job','Job','28','010','10',"He cutteth out rivers among the rocks; and his eye seeth every precious thing."],
['Job','Job','28','011','11',"He bindeth the floods from overflowing; and \emph{the} \emph{thing} \emph{that} \emph{is} hid bringeth he forth to light."],
['Job','Job','28','012','12',"But where shall wisdom be found? and where \emph{is} the place of understanding?"],
['Job','Job','28','013','13',"Man knoweth not the price thereof; neither is it found in the land of the living."],
['Job','Job','28','014','14',"The depth saith, It \emph{is} not in me: and the sea saith, \emph{It} \emph{is} not with me."],
['Job','Job','28','015','15',"It cannot be gotten for gold, neither shall silver be weighed \emph{for} the price thereof."],
['Job','Job','28','016','16',"It cannot be valued with the gold of Ophir, with the precious onyx, or the sapphire."],
['Job','Job','28','017','17',"The gold and the crystal cannot equal it: and the exchange of it \emph{shall} \emph{not} \emph{be} \emph{for} jewels of fine gold."],
['Job','Job','28','018','18',"No mention shall be made of coral, or of pearls: for the price of wisdom \emph{is} above rubies."],
['Job','Job','28','019','19',"The topaz of Ethiopia shall not equal it, neither shall it be valued with pure gold."],
['Job','Job','28','020','20',"Whence then cometh wisdom? and where \emph{is} the place of understanding?"],
['Job','Job','28','021','21',"Seeing it is hid from the eyes of all living, and kept close from the fowls of the air."],
['Job','Job','28','022','22',"Destruction and death say, We have heard the fame thereof with our ears."],
['Job','Job','28','023','23',"God understandeth the way thereof, and he knoweth the place thereof."],
['Job','Job','28','024','24',"For he looketh to the ends of the earth, \emph{and} seeth under the whole heaven;"],
['Job','Job','28','025','25',"To make the weight for the winds; and he weigheth the waters by measure."],
['Job','Job','28','026','26',"When he made a decree for the rain, and a way for the lightning of the thunder:"],
['Job','Job','28','027','27',"Then did he see it, and declare it; he prepared it, yea, and searched it out."],
['Job','Job','28','028','28',"And unto man he said, Behold, the fear of the Lord, that \emph{is} wisdom; and to depart from evil \emph{is} understanding."]
]

Job29 = [
    ['Job','Job','29','001','1',"Moreover Job continued his parable, and said,"],
['Job','Job','29','002','2',"Oh that I were as \emph{in} months past, as \emph{in} the days \emph{when} God preserved me;"],
['Job','Job','29','003','3',"When his candle shined upon my head, \emph{and} \emph{when} by his light I walked \emph{through} darkness;"],
['Job','Job','29','004','4',"As I was in the days of my youth, when the secret of God \emph{was} upon my tabernacle;"],
['Job','Job','29','005','5',"When the Almighty \emph{was} yet with me, \emph{when} my children \emph{were} about me;"],
['Job','Job','29','006','6',"When I washed my steps with butter, and the rock poured me out rivers of oil;"],
['Job','Job','29','007','7',"When I went out to the gate through the city, \emph{when} I prepared my seat in the street!"],
['Job','Job','29','008','8',"The young men saw me, and hid themselves: and the aged arose, \emph{and} stood up."],
['Job','Job','29','009','9',"The princes refrained talking, and laid \emph{their} hand on their mouth."],
['Job','Job','29','010','10',"The nobles held their peace, and their tongue cleaved to the roof of their mouth."],
['Job','Job','29','011','11',"When the ear heard \emph{me}, then it blessed me; and when the eye saw \emph{me}, it gave witness to me:"],
['Job','Job','29','012','12',"Because I delivered the poor that cried, and the fatherless, and \emph{him} \emph{that} \emph{had} none to help him."],
['Job','Job','29','013','13',"The blessing of him that was ready to perish came upon me: and I caused the widow's heart to sing for joy."],
['Job','Job','29','014','14',"I put on righteousness, and it clothed me: my judgment \emph{was} as a robe and a diadem."],
['Job','Job','29','015','15',"I was eyes to the blind, and feet \emph{was} I to the lame."],
['Job','Job','29','016','16',"I \emph{was} a father to the poor: and the cause \emph{which} I knew not I searched out."],
['Job','Job','29','017','17',"And I brake the jaws of the wicked, and plucked the spoil out of his teeth."],
['Job','Job','29','018','18',"Then I said, I shall die in my nest, and I shall multiply \emph{my} days as the sand."],
['Job','Job','29','019','19',"My root \emph{was} spread out by the waters, and the dew lay all night upon my branch."],
['Job','Job','29','020','20',"My glory \emph{was} fresh in me, and my bow was renewed in my hand."],
['Job','Job','29','021','21',"Unto me \emph{men} gave ear, and waited, and kept silence at my counsel."],
['Job','Job','29','022','22',"After my words they spake not again; and my speech dropped upon them."],
['Job','Job','29','023','23',"And they waited for me as for the rain; and they opened their mouth wide \emph{as} for the latter rain."],
['Job','Job','29','024','24',"\emph{If} I laughed on them, they believed \emph{it} not; and the light of my countenance they cast not down."],
['Job','Job','29','025','25',"I chose out their way, and sat chief, and dwelt as a king in the army, as one \emph{that} comforteth the mourners."]
]

Job30 = [
    ['Job','Job','30','001','1',"But now \emph{they} \emph{that} \emph{are} younger than I have me in derision, whose fathers I would have disdained to have set with the dogs of my flock."],
['Job','Job','30','002','2',"Yea, whereto \emph{might} the strength of their hands \emph{profit} me, in whom old age was perished?"],
['Job','Job','30','003','3',"For want and famine \emph{they} \emph{were} solitary; fleeing into the wilderness in former time desolate and waste."],
['Job','Job','30','004','4',"Who cut up mallows by the bushes, and juniper roots \emph{for} their meat."],
['Job','Job','30','005','5',"They were driven forth from among \emph{men}, (they cried after them as \emph{after} a thief;)"],
['Job','Job','30','006','6',"To dwell in the clifts of the valleys, \emph{in} caves of the earth, and \emph{in} the rocks."],
['Job','Job','30','007','7',"Among the bushes they brayed; under the nettles they were gathered together."],
['Job','Job','30','008','8',"\emph{They} \emph{were} children of fools, yea, children of base men: they were viler than the earth."],
['Job','Job','30','009','9',"And now am I their song, yea, I am their byword."],
['Job','Job','30','010','10',"They abhor me, they flee far from me, and spare not to spit in my face."],
['Job','Job','30','011','11',"Because he hath loosed my cord, and afflicted me, they have also let loose the bridle before me."],
['Job','Job','30','012','12',"Upon \emph{my} right \emph{hand} rise the youth; they push away my feet, and they raise up against me the ways of their destruction."],
['Job','Job','30','013','13',"They mar my path, they set forward my calamity, they have no helper."],
['Job','Job','30','014','14',"They came \emph{upon} \emph{me} as a wide breaking in \emph{of} \emph{waters}: in the desolation they rolled themselves \emph{upon} \emph{me}."],
['Job','Job','30','015','15',"Terrors are turned upon me: they pursue my soul as the wind: and my welfare passeth away as a cloud."],
['Job','Job','30','016','16',"And now my soul is poured out upon me; the days of affliction have taken hold upon me."],
['Job','Job','30','017','17',"My bones are pierced in me in the night season: and my sinews take no rest."],
['Job','Job','30','018','18',"By the great force \emph{of} \emph{my} \emph{disease} is my garment changed: it bindeth me about as the collar of my coat."],
['Job','Job','30','019','19',"He hath cast me into the mire, and I am become like dust and ashes."],
['Job','Job','30','020','20',"I cry unto thee, and thou dost not hear me: I stand up, and thou regardest me \emph{not}."],
['Job','Job','30','021','21',"Thou art become cruel to me: with thy strong hand thou opposest thyself against me."],
['Job','Job','30','022','22',"Thou liftest me up to the wind; thou causest me to ride \emph{upon} \emph{it}, and dissolvest my substance."],
['Job','Job','30','023','23',"For I know \emph{that} thou wilt bring me \emph{to} death, and \emph{to} the house appointed for all living."],
['Job','Job','30','024','24',"Howbeit he will not stretch out \emph{his} hand to the grave, though they cry in his destruction."],
['Job','Job','30','025','25',"Did not I weep for him that was in trouble? was \emph{not} my soul grieved for the poor?"],
['Job','Job','30','026','26',"When I looked for good, then evil came \emph{unto} \emph{me}: and when I waited for light, there came darkness."],
['Job','Job','30','027','27',"My bowels boiled, and rested not: the days of affliction prevented me."],
['Job','Job','30','028','28',"I went mourning without the sun: I stood up, \emph{and} I cried in the congregation."],
['Job','Job','30','029','29',"I am a brother to dragons, and a companion to owls."],
['Job','Job','30','030','30',"My skin is black upon me, and my bones are burned with heat."],
['Job','Job','30','031','31',"My harp also is \emph{turned} to mourning, and my organ into the voice of them that weep."]
]

Job31 = [
    ['Job','Job','31','001','1',"I made a covenant with mine eyes; why then should I think upon a maid?"],
['Job','Job','31','002','2',"For what portion of God \emph{is} \emph{there} from above? and \emph{what} inheritance of the Almighty from on high?"],
['Job','Job','31','003','3',"\emph{Is} not destruction to the wicked? and a strange \emph{punishment} to the workers of iniquity?"],
['Job','Job','31','004','4',"Doth not he see my ways, and count all my steps?"],
['Job','Job','31','005','5',"If I have walked with vanity, or if my foot hath hasted to deceit;"],
['Job','Job','31','006','6',"Let me be weighed in an even balance, that God may know mine integrity."],
['Job','Job','31','007','7',"If my step hath turned out of the way, and mine heart walked after mine eyes, and if any blot hath cleaved to mine hands;"],
['Job','Job','31','008','8',"\emph{Then} let me sow, and let another eat; yea, let my offspring be rooted out."],
['Job','Job','31','009','9',"If mine heart have been deceived by a woman, or \emph{if} I have laid wait at my neighbour's door;"],
['Job','Job','31','010','10',"\emph{Then} let my wife grind unto another, and let others bow down upon her."],
['Job','Job','31','011','11',"For this \emph{is} an heinous crime; yea, it \emph{is} an iniquity \emph{to} \emph{be} \emph{punished} \emph{by} the judges."],
['Job','Job','31','012','12',"For it \emph{is} a fire \emph{that} consumeth to destruction, and would root out all mine increase."],
['Job','Job','31','013','13',"If I did despise the cause of my manservant or of my maidservant, when they contended with me;"],
['Job','Job','31','014','14',"What then shall I do when God riseth up? and when he visiteth, what shall I answer him?"],
['Job','Job','31','015','15',"Did not he that made me in the womb make him? and did not one fashion us in the womb?"],
['Job','Job','31','016','16',"If I have withheld the poor from \emph{their} desire, or have caused the eyes of the widow to fail;"],
['Job','Job','31','017','17',"Or have eaten my morsel myself alone, and the fatherless hath not eaten thereof;"],
['Job','Job','31','018','18',"(For from my youth he was brought up with me, as \emph{with} a father, and I have guided her from my mother's womb;)"],
['Job','Job','31','019','19',"If I have seen any perish for want of clothing, or any poor without covering;"],
['Job','Job','31','020','20',"If his loins have not blessed me, and \emph{if} he were \emph{not} warmed with the fleece of my sheep;"],
['Job','Job','31','021','21',"If I have lifted up my hand against the fatherless, when I saw my help in the gate:"],
['Job','Job','31','022','22',"\emph{Then} let mine arm fall from my shoulder blade, and mine arm be broken from the bone."],
['Job','Job','31','023','23',"For destruction \emph{from} God \emph{was} a terror to me, and by reason of his highness I could not endure."],
['Job','Job','31','024','24',"If I have made gold my hope, or have said to the fine gold, \emph{Thou} \emph{art} my confidence;"],
['Job','Job','31','025','25',"If I rejoiced because my wealth \emph{was} great, and because mine hand had gotten much;"],
['Job','Job','31','026','26',"If I beheld the sun when it shined, or the moon walking \emph{in} brightness;"],
['Job','Job','31','027','27',"And my heart hath been secretly enticed, or my mouth hath kissed my hand:"],
['Job','Job','31','028','28',"This also \emph{were} an iniquity \emph{to} \emph{be} \emph{punished} \emph{by} the judge: for I should have denied the God \emph{that} \emph{is} above."],
['Job','Job','31','029','29',"If I rejoiced at the destruction of him that hated me, or lifted up myself when evil found him:"],
['Job','Job','31','030','30',"Neither have I suffered my mouth to sin by wishing a curse to his soul."],
['Job','Job','31','031','31',"If the men of my tabernacle said not, Oh that we had of his flesh! we cannot be satisfied."],
['Job','Job','31','032','32',"The stranger did not lodge in the street: \emph{but} I opened my doors to the traveller."],
['Job','Job','31','033','33',"If I covered my transgressions as Adam, by hiding mine iniquity in my bosom:"],
['Job','Job','31','034','34',"Did I fear a great multitude, or did the contempt of families terrify me, that I kept silence, \emph{and} went not out of the door?"],
['Job','Job','31','035','35',"Oh that one would hear me! behold, my desire \emph{is,} \emph{that} the Almighty would answer me, and \emph{that} mine adversary had written a book."],
['Job','Job','31','036','36',"Surely I would take it upon my shoulder, \emph{and} bind it \emph{as} a crown to me."],
['Job','Job','31','037','37',"I would declare unto him the number of my steps; as a prince would I go near unto him."],
['Job','Job','31','038','38',"If my land cry against me, or that the furrows likewise thereof complain;"],
['Job','Job','31','039','39',"If I have eaten the fruits thereof without money, or have caused the owners thereof to lose their life:"],
['Job','Job','31','040','40',"Let thistles grow instead of wheat, and cockle instead of barley. The words of Job are ended. My harp also is \emph{turned} to mourning, and my organ into the voice of them that weep."]
]

Job32 = [
    ['Job','Job','32','001','1',"So these three men ceased to answer Job, because he \emph{was} righteous in his own eyes."],
['Job','Job','32','002','2',"Then was kindled the wrath of Elihu the son of Barachel the Buzite, of the kindred of Ram: against Job was his wrath kindled, because he justified himself rather than God."],
['Job','Job','32','003','3',"Also against his three friends was his wrath kindled, because they had found no answer, and \emph{yet} had condemned Job."],
['Job','Job','32','004','4',"Now Elihu had waited till Job had spoken, because they \emph{were} elder than he."],
['Job','Job','32','005','5',"When Elihu saw that \emph{there} \emph{was} no answer in the mouth of \emph{these} three men, then his wrath was kindled."],
['Job','Job','32','006','6',"And Elihu the son of Barachel the Buzite answered and said, I \emph{am} young, and ye \emph{are} very old; wherefore I was afraid, and durst not shew you mine opinion."],
['Job','Job','32','007','7',"I said, Days should speak, and multitude of years should teach wisdom."],
['Job','Job','32','008','8',"But \emph{there} \emph{is} a spirit in man: and the inspiration of the Almighty giveth them understanding."],
['Job','Job','32','009','9',"Great men are not \emph{always} wise: neither do the aged understand judgment."],
['Job','Job','32','010','10',"Therefore I said, Hearken to me; I also will shew mine opinion."],
['Job','Job','32','011','11',"Behold, I waited for your words; I gave ear to your reasons, whilst ye searched out what to say."],
['Job','Job','32','012','12',"Yea, I attended unto you, and, behold, \emph{there} \emph{was} none of you that convinced Job, \emph{or} that answered his words:"],
['Job','Job','32','013','13',"Lest ye should say, We have found out wisdom: God thrusteth him down, not man."],
['Job','Job','32','014','14',"Now he hath not directed \emph{his} words against me: neither will I answer him with your speeches."],
['Job','Job','32','015','15',"They were amazed, they answered no more: they left off speaking."],
['Job','Job','32','016','16',"When I had waited, (for they spake not, but stood still, \emph{and} answered no more;)"],
['Job','Job','32','017','17',"\emph{I} \emph{said}, I will answer also my part, I also will shew mine opinion."],
['Job','Job','32','018','18',"For I am full of matter, the spirit within me constraineth me."],
['Job','Job','32','019','19',"Behold, my belly \emph{is} as wine \emph{which} hath no vent; it is ready to burst like new bottles."],
['Job','Job','32','020','20',"I will speak, that I may be refreshed: I will open my lips and answer."],
['Job','Job','32','021','21',"Let me not, I pray you, accept any man's person, neither let me give flattering titles unto man."],
['Job','Job','32','022','22',"For I know not to give flattering titles; \emph{in} \emph{so} \emph{doing} my maker would soon take me away."]
]

Job33 = [
    ['Job','Job','33','001','1',"Wherefore, Job, I pray thee, hear my speeches, and hearken to all my words."],
['Job','Job','33','002','2',"Behold, now I have opened my mouth, my tongue hath spoken in my mouth."],
['Job','Job','33','003','3',"My words \emph{shall} \emph{be} \emph{of} the uprightness of my heart: and my lips shall utter knowledge clearly."],
['Job','Job','33','004','4',"The Spirit of God hath made me, and the breath of the Almighty hath given me life."],
['Job','Job','33','005','5',"If thou canst answer me, set \emph{thy} \emph{words} in order before me, stand up."],
['Job','Job','33','006','6',"Behold, I \emph{am} according to thy wish in God's stead: I also am formed out of the clay."],
['Job','Job','33','007','7',"Behold, my terror shall not make thee afraid, neither shall my hand be heavy upon thee."],
['Job','Job','33','008','8',"Surely thou hast spoken in mine hearing, and I have heard the voice of \emph{thy} words, \emph{saying},"],
['Job','Job','33','009','9',"I am clean without transgression, I \emph{am} innocent; neither \emph{is} \emph{there} iniquity in me."],
['Job','Job','33','010','10',"Behold, he findeth occasions against me, he counteth me for his enemy,"],
['Job','Job','33','011','11',"He putteth my feet in the stocks, he marketh all my paths."],
['Job','Job','33','012','12',"Behold, \emph{in} this thou art not just: I will answer thee, that God is greater than man."],
['Job','Job','33','013','13',"Why dost thou strive against him? for he giveth not account of any of his matters."],
['Job','Job','33','014','14',"For God speaketh once, yea twice, \emph{yet} \emph{man} perceiveth it not."],
['Job','Job','33','015','15',"In a dream, in a vision of the night, when deep sleep falleth upon men, in slumberings upon the bed;"],
['Job','Job','33','016','16',"Then he openeth the ears of men, and sealeth their instruction,"],
['Job','Job','33','017','17',"That he may withdraw man \emph{from} \emph{his} purpose, and hide pride from man."],
['Job','Job','33','018','18',"He keepeth back his soul from the pit, and his life from perishing by the sword."],
['Job','Job','33','019','19',"He is chastened also with pain upon his bed, and the multitude of his bones with strong \emph{pain}:"],
['Job','Job','33','020','20',"So that his life abhorreth bread, and his soul dainty meat."],
['Job','Job','33','021','21',"His flesh is consumed away, that it cannot be seen; and his bones \emph{that} were not seen stick out."],
['Job','Job','33','022','22',"Yea, his soul draweth near unto the grave, and his life to the destroyers."],
['Job','Job','33','023','23',"If there be a messenger with him, an interpreter, one among a thousand, to shew unto man his uprightness:"],
['Job','Job','33','024','24',"Then he is gracious unto him, and saith, Deliver him from going down to the pit: I have found a ransom."],
['Job','Job','33','025','25',"His flesh shall be fresher than a child's: he shall return to the days of his youth:"],
['Job','Job','33','026','26',"He shall pray unto God, and he will be favourable unto him: and he shall see his face with joy: for he will render unto man his righteousness."],
['Job','Job','33','027','27',"He looketh upon men, and \emph{if} \emph{any} say, I have sinned, and perverted \emph{that} \emph{which} \emph{was} right, and it profited me not;"],
['Job','Job','33','028','28',"He will deliver his soul from going into the pit, and his life shall see the light."],
['Job','Job','33','029','29',"Lo, all these \emph{things} worketh God oftentimes with man,"],
['Job','Job','33','030','30',"To bring back his soul from the pit, to be enlightened with the light of the living."],
['Job','Job','33','031','31',"Mark well, O Job, hearken unto me: hold thy peace, and I will speak."],
['Job','Job','33','032','32',"If thou hast any thing to say, answer me: speak, for I desire to justify thee."],
['Job','Job','33','033','33',"If not, hearken unto me: hold thy peace, and I shall teach thee wisdom."]
]

Job34 = [
    ['Job','Job','34','001','1',"Furthermore Elihu answered and said,"],
['Job','Job','34','002','2',"Hear my words, O ye wise \emph{men}; and give ear unto me, ye that have knowledge."],
['Job','Job','34','003','3',"For the ear trieth words, as the mouth tasteth meat."],
['Job','Job','34','004','4',"Let us choose to us judgment: let us know among ourselves what \emph{is} good."],
['Job','Job','34','005','5',"For Job hath said, I am righteous: and God hath taken away my judgment."],
['Job','Job','34','006','6',"Should I lie against my right? my wound \emph{is} incurable without transgression."],
['Job','Job','34','007','7',"What man \emph{is} like Job, \emph{who} drinketh up scorning like water?"],
['Job','Job','34','008','8',"Which goeth in company with the workers of iniquity, and walketh with wicked men."],
['Job','Job','34','009','9',"For he hath said, It profiteth a man nothing that he should delight himself with God."],
['Job','Job','34','010','10',"Therefore hearken unto me, ye men of understanding: far be it from God, \emph{that} \emph{he} \emph{should} \emph{do} wickedness; and \emph{from} the Almighty, \emph{that} \emph{he} \emph{should} \emph{commit} iniquity."],
['Job','Job','34','011','11',"For the work of a man shall he render unto him, and cause every man to find according to \emph{his} ways."],
['Job','Job','34','012','12',"Yea, surely God will not do wickedly, neither will the Almighty pervert judgment."],
['Job','Job','34','013','13',"Who hath given him a charge over the earth? or who hath disposed the whole world?"],
['Job','Job','34','014','14',"If he set his heart upon man, \emph{if} he gather unto himself his spirit and his breath;"],
['Job','Job','34','015','15',"All flesh shall perish together, and man shall turn again unto dust."],
['Job','Job','34','016','16',"If now \emph{thou} \emph{hast} understanding, hear this: hearken to the voice of my words."],
['Job','Job','34','017','17',"Shall even he that hateth right govern? and wilt thou condemn him that is most just?"],
['Job','Job','34','018','18',"\emph{Is} \emph{it} \emph{fit} to say to a king, \emph{Thou} \emph{art} wicked? \emph{and} to princes, \emph{Ye} \emph{are} ungodly?"],
['Job','Job','34','019','19',"\emph{How} \emph{much} \emph{less} \emph{to} \emph{him} that accepteth not the persons of princes, nor regardeth the rich more than the poor? for they all \emph{are} the work of his hands."],
['Job','Job','34','020','20',"In a moment shall they die, and the people shall be troubled at midnight, and pass away: and the mighty shall be taken away without hand."],
['Job','Job','34','021','21',"For his eyes \emph{are} upon the ways of man, and he seeth all his goings."],
['Job','Job','34','022','22',"\emph{There} \emph{is} no darkness, nor shadow of death, where the workers of iniquity may hide themselves."],
['Job','Job','34','023','23',"For he will not lay upon man more \emph{than} \emph{right}; that he should enter into judgment with God."],
['Job','Job','34','024','24',"He shall break in pieces mighty men without number, and set others in their stead."],
['Job','Job','34','025','25',"Therefore he knoweth their works, and he overturneth \emph{them} in the night, so that they are destroyed."],
['Job','Job','34','026','26',"He striketh them as wicked men in the open sight of others;"],
['Job','Job','34','027','27',"Because they turned back from him, and would not consider any of his ways:"],
['Job','Job','34','028','28',"So that they cause the cry of the poor to come unto him, and he heareth the cry of the afflicted."],
['Job','Job','34','029','29',"When he giveth quietness, who then can make trouble? and when he hideth \emph{his} face, who then can behold him? whether \emph{it} \emph{be} \emph{done} against a nation, or against a man only:"],
['Job','Job','34','030','30',"That the hypocrite reign not, lest the people be ensnared."],
['Job','Job','34','031','31',"Surely it is meet to be said unto God, I have borne \emph{chastisement}, I will not offend \emph{any} \emph{more}:"],
['Job','Job','34','032','32',"\emph{That} \emph{which} I see not teach thou me: if I have done iniquity, I will do no more."],
['Job','Job','34','033','33',"\emph{Should} \emph{it} \emph{be} according to thy mind? he will recompense it, whether thou refuse, or whether thou choose; and not I: therefore speak what thou knowest."],
['Job','Job','34','034','34',"Let men of understanding tell me, and let a wise man hearken unto me."],
['Job','Job','34','035','35',"Job hath spoken without knowledge, and his words \emph{were} without wisdom."],
['Job','Job','34','036','36',"My desire \emph{is} \emph{that} Job may be tried unto the end because of \emph{his} answers for wicked men."],
['Job','Job','34','037','37',"For he addeth rebellion unto his sin, he clappeth \emph{his} \emph{hands} among us, and multiplieth his words against God."]
]

Job35 = [
    ['Job','Job','35','001','1',"Elihu spake moreover, and said,"],
['Job','Job','35','002','2',"Thinkest thou this to be right, \emph{that} thou saidst, My righteousness \emph{is} more than God's?"],
['Job','Job','35','003','3',"For thou saidst, What advantage will it be unto thee? \emph{and}, What profit shall I have, \emph{if} \emph{I} \emph{be} \emph{cleansed} from my sin?"],
['Job','Job','35','004','4',"I will answer thee, and thy companions with thee."],
['Job','Job','35','005','5',"Look unto the heavens, and see; and behold the clouds \emph{which} are higher than thou."],
['Job','Job','35','006','6',"If thou sinnest, what doest thou against him? or \emph{if} thy transgressions be multiplied, what doest thou unto him?"],
['Job','Job','35','007','7',"If thou be righteous, what givest thou him? or what receiveth he of thine hand?"],
['Job','Job','35','008','8',"Thy wickedness \emph{may} \emph{hurt} a man as thou \emph{art}; and thy righteousness \emph{may} \emph{profit} the son of man."],
['Job','Job','35','009','9',"By reason of the multitude of oppressions they make \emph{the} \emph{oppressed} to cry: they cry out by reason of the arm of the mighty."],
['Job','Job','35','010','10',"But none saith, Where \emph{is} God my maker, who giveth songs in the night;"],
['Job','Job','35','011','11',"Who teacheth us more than the beasts of the earth, and maketh us wiser than the fowls of heaven?"],
['Job','Job','35','012','12',"There they cry, but none giveth answer, because of the pride of evil men."],
['Job','Job','35','013','13',"Surely God will not hear vanity, neither will the Almighty regard it."],
['Job','Job','35','014','14',"Although thou sayest thou shalt not see him, \emph{yet} judgment \emph{is} before him; therefore trust thou in him."],
['Job','Job','35','015','15',"But now, because \emph{it} \emph{is} not \emph{so}, he hath visited in his anger; yet he knoweth \emph{it} not in great extremity:"],
['Job','Job','35','016','16',"Therefore doth Job open his mouth in vain; he multiplieth words without knowledge."]
]

Job36 = [
['Job','Job','36','001','1',"Elihu also proceeded, and said,"],
['Job','Job','36','002','2',"Suffer me a little, and I will shew thee that \emph{I} \emph{have} yet to speak on God's behalf."],
['Job','Job','36','003','3',"I will fetch my knowledge from afar, and will ascribe righteousness to my Maker."],
['Job','Job','36','004','4',"For truly my words \emph{shall} not \emph{be} false: he that is perfect in knowledge \emph{is} with thee."],
['Job','Job','36','005','5',"Behold, God \emph{is} mighty, and despiseth not \emph{any:} \emph{he} \emph{is} mighty in strength \emph{and} wisdom."],
['Job','Job','36','006','6',"He preserveth not the life of the wicked: but giveth right to the poor."],
['Job','Job','36','007','7',"He withdraweth not his eyes from the righteous: but with kings \emph{are} \emph{they} on the throne; yea, he doth establish them for ever, and they are exalted."],
['Job','Job','36','008','8',"And if \emph{they} \emph{be} bound in fetters, \emph{and} be holden in cords of affliction;"],
['Job','Job','36','009','9',"Then he sheweth them their work, and their transgressions that they have exceeded."],
['Job','Job','36','010','10',"He openeth also their ear to discipline, and commandeth that they return from iniquity."],
['Job','Job','36','011','11',"If they obey and serve \emph{him}, they shall spend their days in prosperity, and their years in pleasures."],
['Job','Job','36','012','12',"But if they obey not, they shall perish by the sword, and they shall die without knowledge."],
['Job','Job','36','013','13',"But the hypocrites in heart heap up wrath: they cry not when he bindeth them."],
['Job','Job','36','014','14',"They die in youth, and their life \emph{is} among the unclean."],
['Job','Job','36','015','15',"He delivereth the poor in his affliction, and openeth their ears in oppression."],
['Job','Job','36','016','16',"Even so would he have removed thee out of the strait \emph{into} a broad place, where \emph{there} \emph{is} no straitness; and that which should be set on thy table \emph{should} \emph{be} full of fatness."],
['Job','Job','36','017','17',"But thou hast fulfilled the judgment of the wicked: judgment and justice take hold \emph{on} \emph{thee}."],
['Job','Job','36','018','18',"Because \emph{there} \emph{is} wrath, \emph{beware} lest he take thee away with \emph{his} stroke: then a great ransom cannot deliver thee."],
['Job','Job','36','019','19',"Will he esteem thy riches? \emph{no}, not gold, nor all the forces of strength."],
['Job','Job','36','020','20',"Desire not the night, when people are cut off in their place."],
['Job','Job','36','021','21',"Take heed, regard not iniquity: for this hast thou chosen rather than affliction."],
['Job','Job','36','022','22',"Behold, God exalteth by his power: who teacheth like him?"],
['Job','Job','36','023','23',"Who hath enjoined him his way? or who can say, Thou hast wrought iniquity?"],
['Job','Job','36','024','24',"Remember that thou magnify his work, which men behold."],
['Job','Job','36','025','25',"Every man may see it; man may behold \emph{it} afar off."],
['Job','Job','36','026','26',"Behold, God \emph{is} great, and we know \emph{him} not, neither can the number of his years be searched out."],
['Job','Job','36','027','27',"For he maketh small the drops of water: they pour down rain according to the vapour thereof:"],
['Job','Job','36','028','28',"Which the clouds do drop \emph{and} distil upon man abundantly."],
['Job','Job','36','029','29',"Also can \emph{any} understand the spreadings of the clouds, \emph{or} the noise of his tabernacle?"],
['Job','Job','36','030','30',"Behold, he spreadeth his light upon it, and covereth the bottom of the sea."],
['Job','Job','36','031','31',"For by them judgeth he the people; he giveth meat in abundance."],
['Job','Job','36','032','32',"With clouds he covereth the light; and commandeth it \emph{not} \emph{to} \emph{shine} by \emph{the} \emph{cloud} that cometh betwixt."],
['Job','Job','36','033','33',"The noise thereof sheweth concerning it, the cattle also concerning the vapour."]
]

Job37 = [
    ['Job','Job','37','001','1',"At this also my heart trembleth, and is moved out of his place."],
['Job','Job','37','002','2',"Hear attentively the noise of his voice, and the sound \emph{that} goeth out of his mouth."],
['Job','Job','37','003','3',"He directeth it under the whole heaven, and his lightning unto the ends of the earth."],
['Job','Job','37','004','4',"After it a voice roareth: he thundereth with the voice of his excellency; and he will not stay them when his voice is heard."],
['Job','Job','37','005','5',"God thundereth marvellously with his voice; great things doeth he, which we cannot comprehend."],
['Job','Job','37','006','6',"For he saith to the snow, Be thou \emph{on} the earth; likewise to the small rain, and to the great rain of his strength."],
['Job','Job','37','007','7',"He sealeth up the hand of every man; that all men may know his work."],
['Job','Job','37','008','8',"Then the beasts go into dens, and remain in their places."],
['Job','Job','37','009','9',"Out of the south cometh the whirlwind: and cold out of the north."],
['Job','Job','37','010','10',"By the breath of God frost is given: and the breadth of the waters is straitened."],
['Job','Job','37','011','11',"Also by watering he wearieth the thick cloud: he scattereth his bright cloud:"],
['Job','Job','37','012','12',"And it is turned round about by his counsels: that they may do whatsoever he commandeth them upon the face of the world in the earth."],
['Job','Job','37','013','13',"He causeth it to come, whether for correction, or for his land, or for mercy."],
['Job','Job','37','014','14',"Hearken unto this, O Job: stand still, and consider the wondrous works of God."],
['Job','Job','37','015','15',"Dost thou know when God disposed them, and caused the light of his cloud to shine?"],
['Job','Job','37','016','16',"Dost thou know the balancings of the clouds, the wondrous works of him which is perfect in knowledge?"],
['Job','Job','37','017','17',"How thy garments \emph{are} warm, when he quieteth the earth by the south \emph{wind}?"],
['Job','Job','37','018','18',"Hast thou with him spread out the sky, \emph{which} \emph{is} strong, \emph{and} as a molten looking glass?"],
['Job','Job','37','019','19',"Teach us what we shall say unto him; \emph{for} we cannot order \emph{our} \emph{speech} by reason of darkness."],
['Job','Job','37','020','20',"Shall it be told him that I speak? if a man speak, surely he shall be swallowed up."],
['Job','Job','37','021','21',"And now \emph{men} see not the bright light which \emph{is} in the clouds: but the wind passeth, and cleanseth them."],
['Job','Job','37','022','22',"Fair weather cometh out of the north: with God \emph{is} terrible majesty."],
['Job','Job','37','023','23',"\emph{Touching} the Almighty, we cannot find him out: \emph{he} \emph{is} excellent in power, and in judgment, and in plenty of justice: he will not afflict."],
['Job','Job','37','024','24',"Men do therefore fear him: he respecteth not any \emph{that} \emph{are} wise of heart."]
]

Job38 = [
    ['Job','Job','38','001','1',"Then the LORD answered Job out of the whirlwind, and said,"],
['Job','Job','38','002','2',"Who \emph{is} this that darkeneth counsel by words without knowledge?"],
['Job','Job','38','003','3',"Gird up now thy loins like a man; for I will demand of thee, and answer thou me."],
['Job','Job','38','004','4',"Where wast thou when I laid the foundations of the earth? declare, if thou hast understanding."],
['Job','Job','38','005','5',"Who hath laid the measures thereof, if thou knowest? or who hath stretched the line upon it?"],
['Job','Job','38','006','6',"Whereupon are the foundations thereof fastened? or who laid the corner stone thereof;"],
['Job','Job','38','007','7',"When the morning stars sang together, and all the sons of God shouted for joy?"],
['Job','Job','38','008','8',"Or \emph{who} shut up the sea with doors, when it brake forth, \emph{as} \emph{if} it had issued out of the womb?"],
['Job','Job','38','009','9',"When I made the cloud the garment thereof, and thick darkness a swaddlingband for it,"],
['Job','Job','38','010','10',"And brake up for it my decreed \emph{place}, and set bars and doors,"],
['Job','Job','38','011','11',"And said, Hitherto shalt thou come, but no further: and here shall thy proud waves be stayed?"],
['Job','Job','38','012','12',"Hast thou commanded the morning since thy days; \emph{and} caused the dayspring to know his place;"],
['Job','Job','38','013','13',"That it might take hold of the ends of the earth, that the wicked might be shaken out of it?"],
['Job','Job','38','014','14',"It is turned as clay \emph{to} the seal; and they stand as a garment."],
['Job','Job','38','015','15',"And from the wicked their light is withholden, and the high arm shall be broken."],
['Job','Job','38','016','16',"Hast thou entered into the springs of the sea? or hast thou walked in the search of the depth?"],
['Job','Job','38','017','17',"Have the gates of death been opened unto thee? or hast thou seen the doors of the shadow of death?"],
['Job','Job','38','018','18',"Hast thou perceived the breadth of the earth? declare if thou knowest it all."],
['Job','Job','38','019','19',"Where \emph{is} the way \emph{where} light dwelleth? and \emph{as} \emph{for} darkness, where \emph{is} the place thereof,"],
['Job','Job','38','020','20',"That thou shouldest take it to the bound thereof, and that thou shouldest know the paths \emph{to} the house thereof?"],
['Job','Job','38','021','21',"Knowest thou \emph{it}, because thou wast then born? or \emph{because} the number of thy days \emph{is} great?"],
['Job','Job','38','022','22',"Hast thou entered into the treasures of the snow? or hast thou seen the treasures of the hail,"],
['Job','Job','38','023','23',"Which I have reserved against the time of trouble, against the day of battle and war?"],
['Job','Job','38','024','24',"By what way is the light parted, \emph{which} scattereth the east wind upon the earth?"],
['Job','Job','38','025','25',"Who hath divided a watercourse for the overflowing of waters, or a way for the lightning of thunder;"],
['Job','Job','38','026','26',"To cause it to rain on the earth, \emph{where} no man \emph{is;} \emph{on} the wilderness, wherein \emph{there} \emph{is} no man;"],
['Job','Job','38','027','27',"To satisfy the desolate and waste \emph{ground}; and to cause the bud of the tender herb to spring forth?"],
['Job','Job','38','028','28',"Hath the rain a father? or who hath begotten the drops of dew?"],
['Job','Job','38','029','29',"Out of whose womb came the ice? and the hoary frost of heaven, who hath gendered it?"],
['Job','Job','38','030','30',"The waters are hid as \emph{with} a stone, and the face of the deep is frozen."],
['Job','Job','38','031','31',"Canst thou bind the sweet influences of Pleiades, or loose the bands of Orion?"],
['Job','Job','38','032','32',"Canst thou bring forth Mazzaroth in his season? or canst thou guide Arcturus with his sons?"],
['Job','Job','38','033','33',"Knowest thou the ordinances of heaven? canst thou set the dominion thereof in the earth?"],
['Job','Job','38','034','34',"Canst thou lift up thy voice to the clouds, that abundance of waters may cover thee?"],
['Job','Job','38','035','35',"Canst thou send lightnings, that they may go, and say unto thee, Here we \emph{are}?"],
['Job','Job','38','036','36',"Who hath put wisdom in the inward parts? or who hath given understanding to the heart?"],
['Job','Job','38','037','37',"Who can number the clouds in wisdom? or who can stay the bottles of heaven,"],
['Job','Job','38','038','38',"When the dust groweth into hardness, and the clods cleave fast together?"],
['Job','Job','38','039','39',"Wilt thou hunt the prey for the lion? or fill the appetite of the young lions,"],
['Job','Job','38','040','40',"When they couch in \emph{their} dens, \emph{and} abide in the covert to lie in wait?"],
['Job','Job','38','041','41',"Who provideth for the raven his food? when his young ones cry unto God, they wander for lack of meat."]
]

Job39 = [
    ['Job','Job','39','001','1',"Knowest thou the time when the wild goats of the rock bring forth? \emph{or} canst thou mark when the hinds do calve?"],
['Job','Job','39','002','2',"Canst thou number the months \emph{that} they fulfil? or knowest thou the time when they bring forth?"],
['Job','Job','39','003','3',"They bow themselves, they bring forth their young ones, they cast out their sorrows."],
['Job','Job','39','004','4',"Their young ones are in good liking, they grow up with corn; they go forth, and return not unto them."],
['Job','Job','39','005','5',"Who hath sent out the wild ass free? or who hath loosed the bands of the wild ass?"],
['Job','Job','39','006','6',"Whose house I have made the wilderness, and the barren land his dwellings."],
['Job','Job','39','007','7',"He scorneth the multitude of the city, neither regardeth he the crying of the driver."],
['Job','Job','39','008','8',"The range of the mountains \emph{is} his pasture, and he searcheth after every green thing."],
['Job','Job','39','009','9',"Will the unicorn be willing to serve thee, or abide by thy crib?"],
['Job','Job','39','010','10',"Canst thou bind the unicorn with his band in the furrow? or will he harrow the valleys after thee?"],
['Job','Job','39','011','11',"Wilt thou trust him, because his strength \emph{is} great? or wilt thou leave thy labour to him?"],
['Job','Job','39','012','12',"Wilt thou believe him, that he will bring home thy seed, and gather \emph{it} \emph{into} thy barn?"],
['Job','Job','39','013','13',"\emph{Gavest} \emph{thou} the goodly wings unto the peacocks? or wings and feathers unto the ostrich?"],
['Job','Job','39','014','14',"Which leaveth her eggs in the earth, and warmeth them in dust,"],
['Job','Job','39','015','15',"And forgetteth that the foot may crush them, or that the wild beast may break them."],
['Job','Job','39','016','16',"She is hardened against her young ones, as though \emph{they} \emph{were} not hers: her labour is in vain without fear;"],
['Job','Job','39','017','17',"Because God hath deprived her of wisdom, neither hath he imparted to her understanding."],
['Job','Job','39','018','18',"What time she lifteth up herself on high, she scorneth the horse and his rider."],
['Job','Job','39','019','19',"Hast thou given the horse strength? hast thou clothed his neck with thunder?"],
['Job','Job','39','020','20',"Canst thou make him afraid as a grasshopper? the glory of his nostrils \emph{is} terrible."],
['Job','Job','39','021','21',"He paweth in the valley, and rejoiceth in \emph{his} strength: he goeth on to meet the armed men."],
['Job','Job','39','022','22',"He mocketh at fear, and is not affrighted; neither turneth he back from the sword."],
['Job','Job','39','023','23',"The quiver rattleth against him, the glittering spear and the shield."],
['Job','Job','39','024','24',"He swalloweth the ground with fierceness and rage: neither believeth he that \emph{it} \emph{is} the sound of the trumpet."],
['Job','Job','39','025','25',"He saith among the trumpets, Ha, ha; and he smelleth the battle afar off, the thunder of the captains, and the shouting."],
['Job','Job','39','026','26',"Doth the hawk fly by thy wisdom, \emph{and} stretch her wings toward the south?"],
['Job','Job','39','027','27',"Doth the eagle mount up at thy command, and make her nest on high?"],
['Job','Job','39','028','28',"She dwelleth and abideth on the rock, upon the crag of the rock, and the strong place."],
['Job','Job','39','029','29',"From thence she seeketh the prey, \emph{and} her eyes behold afar off."],
['Job','Job','39','030','30',"Her young ones also suck up blood: and where the slain \emph{are}, there \emph{is} she."]
]

Job40 = [
['Job','Job','40','001','1',"Moreover the LORD answered Job, and said,"],
['Job','Job','40','002','2',"Shall he that contendeth with the Almighty instruct \emph{him}? he that reproveth God, let him answer it."],
['Job','Job','40','003','3',"Then Job answered the LORD, and said,"],
['Job','Job','40','004','4',"Behold, I am vile; what shall I answer thee? I will lay mine hand upon my mouth."],
['Job','Job','40','005','5',"Once have I spoken; but I will not answer: yea, twice; but I will proceed no further."],
['Job','Job','40','006','6',"Then answered the LORD unto Job out of the whirlwind, and said,"],
['Job','Job','40','007','7',"Gird up thy loins now like a man: I will demand of thee, and declare thou unto me."],
['Job','Job','40','008','8',"Wilt thou also disannul my judgment? wilt thou condemn me, that thou mayest be righteous?"],
['Job','Job','40','009','9',"Hast thou an arm like God? or canst thou thunder with a voice like him?"],
['Job','Job','40','010','10',"Deck thyself now \emph{with} majesty and excellency; and array thyself with glory and beauty."],
['Job','Job','40','011','11',"Cast abroad the rage of thy wrath: and behold every one \emph{that} \emph{is} proud, and abase him."],
['Job','Job','40','012','12',"Look on every one \emph{that} \emph{is} proud, \emph{and} bring him low; and tread down the wicked in their place."],
['Job','Job','40','013','13',"Hide them in the dust together; \emph{and} bind their faces in secret."],
['Job','Job','40','014','14',"Then will I also confess unto thee that thine own right hand can save thee."],
['Job','Job','40','015','15',"Behold now behemoth, which I made with thee; he eateth grass as an ox."],
['Job','Job','40','016','16',"Lo now, his strength \emph{is} in his loins, and his force \emph{is} in the navel of his belly."],
['Job','Job','40','017','17',"He moveth his tail like a cedar: the sinews of his stones are wrapped together."],
['Job','Job','40','018','18',"His bones \emph{are} \emph{as} strong pieces of brass; his bones \emph{are} like bars of iron."],
['Job','Job','40','019','19',"He \emph{is} the chief of the ways of God: he that made him can make his sword to approach \emph{unto} \emph{him}."],
['Job','Job','40','020','20',"Surely the mountains bring him forth food, where all the beasts of the field play."],
['Job','Job','40','021','21',"He lieth under the shady trees, in the covert of the reed, and fens."],
['Job','Job','40','022','22',"The shady trees cover him \emph{with} their shadow; the willows of the brook compass him about."],
['Job','Job','40','023','23',"Behold, he drinketh up a river, \emph{and} hasteth not: he trusteth that he can draw up Jordan into his mouth."],
['Job','Job','40','024','24',"He taketh it with his eyes: \emph{his} nose pierceth through snares."]
]

Job41 = [
['Job','Job','41','001','1',"Canst thou draw out leviathan with an hook? or his tongue with a cord \emph{which} thou lettest down?"],
['Job','Job','41','002','2',"Canst thou put an hook into his nose? or bore his jaw through with a thorn?"],
['Job','Job','41','003','3',"Will he make many supplications unto thee? will he speak soft \emph{words} unto thee?"],
['Job','Job','41','004','4',"Will he make a covenant with thee? wilt thou take him for a servant for ever?"],
['Job','Job','41','005','5',"Wilt thou play with him as \emph{with} a bird? or wilt thou bind him for thy maidens?"],
['Job','Job','41','006','6',"Shall the companions make a banquet of him? shall they part him among the merchants?"],
['Job','Job','41','007','7',"Canst thou fill his skin with barbed irons? or his head with fish spears?"],
['Job','Job','41','008','8',"Lay thine hand upon him, remember the battle, do no more."],
['Job','Job','41','009','9',"Behold, the hope of him is in vain: shall not \emph{one} be cast down even at the sight of him?"],
['Job','Job','41','010','10',"None \emph{is} \emph{so} fierce that dare stir him up: who then is able to stand before me?"],
['Job','Job','41','011','11',"Who hath prevented me, that I should repay \emph{him?} \emph{whatsoever} \emph{is} under the whole heaven is mine."],
['Job','Job','41','012','12',"I will not conceal his parts, nor his power, nor his comely proportion."],
['Job','Job','41','013','13',"Who can discover the face of his garment? \emph{or} who can come \emph{to} \emph{him} with his double bridle?"],
['Job','Job','41','014','14',"Who can open the doors of his face? his teeth \emph{are} terrible round about."],
['Job','Job','41','015','15',"\emph{His} scales \emph{are} \emph{his} pride, shut up together \emph{as} \emph{with} a close seal."],
['Job','Job','41','016','16',"One is so near to another, that no air can come between them."],
['Job','Job','41','017','17',"They are joined one to another, they stick together, that they cannot be sundered."],
['Job','Job','41','018','18',"By his neesings a light doth shine, and his eyes \emph{are} like the eyelids of the morning."],
['Job','Job','41','019','19',"Out of his mouth go burning lamps, \emph{and} sparks of fire leap out."],
['Job','Job','41','020','20',"Out of his nostrils goeth smoke, as \emph{out} of a seething pot or caldron."],
['Job','Job','41','021','21',"His breath kindleth coals, and a flame goeth out of his mouth."],
['Job','Job','41','022','22',"In his neck remaineth strength, and sorrow is turned into joy before him."],
['Job','Job','41','023','23',"The flakes of his flesh are joined together: they are firm in themselves; they cannot be moved."],
['Job','Job','41','024','24',"His heart is as firm as a stone; yea, as hard as a piece of the nether \emph{millstone}."],
['Job','Job','41','025','25',"When he raiseth up himself, the mighty are afraid: by reason of breakings they purify themselves."],
['Job','Job','41','026','26',"The sword of him that layeth at him cannot hold: the spear, the dart, nor the habergeon."],
['Job','Job','41','027','27',"He esteemeth iron as straw, \emph{and} brass as rotten wood."],
['Job','Job','41','028','28',"The arrow cannot make him flee: slingstones are turned with him into stubble."],
['Job','Job','41','029','29',"Darts are counted as stubble: he laugheth at the shaking of a spear."],
['Job','Job','41','030','30',"Sharp stones \emph{are} under him: he spreadeth sharp pointed things upon the mire."],
['Job','Job','41','031','31',"He maketh the deep to boil like a pot: he maketh the sea like a pot of ointment."],
['Job','Job','41','032','32',"He maketh a path to shine after him; \emph{one} would think the deep \emph{to} \emph{be} hoary."],
['Job','Job','41','033','33',"Upon earth there is not his like, who is made without fear."],
['Job','Job','41','034','34',"He beholdeth all high \emph{things}: he \emph{is} a king over all the children of pride."]
]

Job42 = [
['Job','Job','42','001','1',"Then Job answered the LORD, and said,"],
['Job','Job','42','002','2',"I know that thou canst do every \emph{thing}, and \emph{that} no thought can be withholden from thee."],
['Job','Job','42','003','3',"Who \emph{is} he that hideth counsel without knowledge? therefore have I uttered that I understood not; things too wonderful for me, which I knew not."],
['Job','Job','42','004','4',"Hear, I beseech thee, and I will speak: I will demand of thee, and declare thou unto me."],
['Job','Job','42','005','5',"I have heard of thee by the hearing of the ear: but now mine eye seeth thee."],
['Job','Job','42','006','6',"Wherefore I abhor \emph{myself}, and repent in dust and ashes."],
['Job','Job','42','007','7',"And it was \emph{so}, that after the LORD had spoken these words unto Job, the LORD said to Eliphaz the Temanite, My wrath is kindled against thee, and against thy two friends: for ye have not spoken of me \emph{the} \emph{thing} \emph{that} \emph{is} right, as my servant Job \emph{hath}."],
['Job','Job','42','008','8',"Therefore take unto you now seven bullocks and seven rams, and go to my servant Job, and offer up for yourselves a burnt offering; and my servant Job shall pray for you: for him will I accept: lest I deal with you \emph{after} \emph{your} folly, in that ye have not spoken of me \emph{the} \emph{thing} \emph{which} \emph{is} right, like my servant Job."],
['Job','Job','42','009','9',"So Eliphaz the Temanite and Bildad the Shuhite \emph{and} Zophar the Naamathite went, and did according as the LORD commanded them: the LORD also accepted Job."],
['Job','Job','42','010','10',"And the LORD turned the captivity of Job, when he prayed for his friends: also the LORD gave Job twice as much as he had before."],
['Job','Job','42','011','11',"Then came there unto him all his brethren, and all his sisters, and all they that had been of his acquaintance before, and did eat bread with him in his house: and they bemoaned him, and comforted him over all the evil that the LORD had brought upon him: every man also gave him a piece of money, and every one an earring of gold."],
['Job','Job','42','012','12',"So the LORD blessed the latter end of Job more than his beginning: for he had fourteen thousand sheep, and six thousand camels, and a thousand yoke of oxen, and a thousand she asses."],
['Job','Job','42','013','13',"He had also seven sons and three daughters."],
['Job','Job','42','014','14',"And he called the name of the first, Jemima; and the name of the second, Kezia; and the name of the third, Keren-happuch."],
['Job','Job','42','015','15',"And in all the land were no women found \emph{so} fair as the daughters of Job: and their father gave them inheritance among their brethren."],
['Job','Job','42','016','16',"After this lived Job an hundred and forty years, and saw his sons, and his sons' sons, \emph{even} four generations."],
['Job','Job','42','017','17',"So Job died, \emph{being} old and full of days."]
]





def main():
    newProcessVerses('Job39',Job39)
if __name__ == "__main__":
    main()